In [362]:
import pandas as pd
import numpy as np

In [2]:
### Get all the pillar names from the excel

In [3]:
names = pd.read_excel('../../UNDP Digital Assessment Data Framework Filename Matching V7.xlsx')

In [4]:
col_names = ['Indicator','check', 'Data Source','Data Link','Index','Filename','Sub-Pillar']

In [5]:
names = names[col_names]

In [6]:
names.head()

,Indicator,check,Data Source,Data Link,Index,Filename,Sub-Pillar
0,Countries,NaN,UN Statistics Division: List of Countries,https://unstats.un.org,False,Countries,NaN
1,"Database of Global Administrative Areas (GADM,...",NaN,NaN,https://gadm.org,False,NaN,NaN
2,High Resolution Population Density Maps + Demo...,NaN,NaN,NaN,False,NaN,NaN
3,population density vs openstreetmap object den...,NaN,NaN,NaN,False,NaN,NaN
4,Population Density,Infrastructure,World Bank: World Development Indicators,https://datacatalog.worldbank.org,False,population_density,Connectivity Technology


In [7]:
# get all the files per pillar
data_stats = names.groupby('check').agg({'Filename':'count','Indicator':'count'})

In [8]:
data_stats

,Filename,Indicator
check,,
Business,18,27
Foundations,13,22
Government,10,15
Infrastructure,47,58
People,35,49
Regulation,5,8
Strategy,1,1


In [9]:
### Infrastructure

In [10]:
bnames = names[(names.check=='Infrastructure')&(~names.Filename.isna())]#&(names.Index==False)]
bnames.head(25)

,Indicator,check,Data Source,Data Link,Index,Filename,Sub-Pillar
4,Population Density,Infrastructure,World Bank: World Development Indicators,https://datacatalog.worldbank.org,False,population_density,Connectivity Technology
5,Broadband Density,Infrastructure,ITU: Digital Development Dashboard,https://www.itu.int/itu-d/sites/statistics,False,ITU_database,Connectivity Technology
6,% of population covered by internet connectivity,Infrastructure,ITU: Digital Development Dashboard,https://www.itu.int/itu-d/sites/statistics,False,ITU_database,Connectivity Technology
7,% of population covered by mobile 2G+ data con...,Infrastructure,GSMA: Mobile Connectivity Index,https://www.mobileconnectivityindex.com,False,countries_mobile_connectivity,Connectivity Technology
8,% of population covered by mobile 3G+ data con...,Infrastructure,GSMA: Mobile Connectivity Index,https://www.mobileconnectivityindex.com,False,countries_mobile_connectivity,Connectivity Technology
9,% of population covered by mobile 4G+ data con...,Infrastructure,GSMA: Mobile Connectivity Index,https://www.mobileconnectivityindex.com,False,countries_mobile_connectivity,Connectivity Technology
10,% of population covered by mobile 5G+ data con...,Infrastructure,GSMA: Mobile Connectivity Index,https://www.mobileconnectivityindex.com,False,countries_mobile_connectivity,Connectivity Technology
11,Mobile Coverage Maps,Infrastructure,ITU: Digital Development Dashboard,https://www.itu.int/en/Pages/default.aspx,False,ITU_database,Connectivity Technology
12,Electricity Density,Infrastructure,University of Oxford: Our World in Data,https://ourworldindata.org,False,electricity_density,Connectivity Technology
13,% of population covered by electricity,Infrastructure,World Bank: World Development Indicators,https://datacatalog.worldbank.org,False,population_electricity_coverage,Connectivity Technology


In [11]:
# get list of names for all indicators
indicators = bnames.Indicator.unique()
subpillars = bnames['Sub-Pillar'].unique()

In [12]:
# get all file names
bfiles = bnames.Filename.unique()

In [13]:
bfiles

array(['population_density', 'ITU_database',
       'countries_mobile_connectivity', 'electricity_density',
       'population_electricity_coverage', 'TCdata360',
       'sustainability_index', 'mobile_density', 'e_government_index',
       'spectrum_allocated_mobile_providers', 'countries_ixp',
       'mobile_speed', 'fixed_bdbd_spd_dl_ul', 'internet_speed',
       'mobile_latency', 'postal_coverage', 'logistics_performance_index',
       'national_cybersecurity_index',
       'dice_export_global_cybersecurity_index',
       'software_developer_ecosystem_size',
       'digital_platform_economy_index', 'migration_skill',
       'migration_industry', 'migration_country',
       'global_fintech_ranking', 'tech_hubs', 'banking_sector_size',
       'angel_investment', 'startup_eco_size',
       'international_co_inventions', 'global_innovation_dataset',
       'patent_application_PCT', 'Hague_system'], dtype=object)

In [14]:
subpillars

array(['Connectivity Technology', 'Innovation Ecosystem'], dtype=object)

In [15]:
# formula for converting scale 0-100
def convert_rank(old_value, old_min= 0, old_max= 100, new_min=1, new_max=6 ):
    """ Convert old scale values scale into new scale values"""
    old_range = old_max - old_min
    new_range = new_max - new_min
    new_value = (((old_value-old_min)*new_range)/old_range)+new_min
    return new_value

In [16]:
# formula for converting scale 0-1
def convert_rank_b(old_value, old_min= 0, old_max= 1, new_min=1, new_max=6 ):
    """ Convert old scale values scale into new scale values"""
    old_range = old_max - old_min
    new_range = new_max - new_min
    new_value = (((old_value-old_min)*new_range)/old_range)+new_min
    return new_value

In [17]:
# formula for converting scale 1-5
def convert_rank_c(old_value, old_min= 1, old_max= 5, new_min=1, new_max=6 ):
    """ Convert old scale values scale into new scale values"""
    old_range = old_max - old_min
    new_range = new_max - new_min
    new_value = (((old_value-old_min)*new_range)/old_range)+new_min
    return new_value

In [18]:
# formula for converting scale 1-7
def convert_rank_d(old_value, old_min= 1, old_max= 7, new_min=1, new_max=6 ):
    """ Convert old scale values scale into new scale values"""
    old_range = old_max - old_min
    new_range = new_max - new_min
    new_value = (((old_value-old_min)*new_range)/old_range)+new_min
    return new_value

In [19]:
### 1. Population Density

In [20]:
### 2. Broadband Density

In [21]:
indicators[1]

# load data
indicator = indicators[1]
print(indicator)
bf = bnames[bnames['Indicator']==indicator]['Filename'].values[0]
print(bf)

df = pd.read_csv('../../processed/{}.csv'.format(bf))
df

Broadband Density
ITU_database


,Country,Region,ISO,Indicator name,Year,Value,data_country,data_year
0,Angola,Africa,AGO,Female mobile phone ownership as a % of total ...,2010.0,NaN,NaN,NaN
1,Benin,Africa,BEN,Female mobile phone ownership as a % of total ...,2010.0,NaN,NaN,NaN
2,Botswana,Africa,BWA,Female mobile phone ownership as a % of total ...,2010.0,NaN,NaN,NaN
3,Burkina Faso,Africa,BFA,Female mobile phone ownership as a % of total ...,2010.0,NaN,NaN,NaN
4,Burundi,Africa,BDI,Female mobile phone ownership as a % of total ...,2010.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
47807,Mexico,The Americas,MEX,"Individuals using the Internet, total (%)",2020.0,71.970000,NaN,NaN
47808,Paraguay,The Americas,PRY,"Individuals using the Internet, total (%)",2020.0,74.515240,NaN,NaN
47809,Peru,The Americas,PER,"Individuals using the Internet, total (%)",2020.0,65.251799,NaN,NaN
47810,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
df = df[(df.Year == 2020)]
df = df[(df['Indicator name'] == 'Fixed broadband subscriptions per 100 inhabitants')]
df.rename(columns={'Country':'Country Name'}, inplace=True)
df

,Country Name,Region,ISO,Indicator name,Year,Value,data_country,data_year
36787,Angola,Africa,AGO,Fixed broadband subscriptions per 100 inhabitants,2020.0,0.701662,NaN,NaN
36788,Benin,Africa,BEN,Fixed broadband subscriptions per 100 inhabitants,2020.0,0.247303,NaN,NaN
36789,Botswana,Africa,BWA,Fixed broadband subscriptions per 100 inhabitants,2020.0,3.057373,NaN,NaN
36790,Burkina Faso,Africa,BFA,Fixed broadband subscriptions per 100 inhabitants,2020.0,0.066875,NaN,NaN
36791,Burundi,Africa,BDI,Fixed broadband subscriptions per 100 inhabitants,2020.0,0.035574,NaN,NaN
...,...,...,...,...,...,...,...,...
36978,Suriname,The Americas,SUR,Fixed broadband subscriptions per 100 inhabitants,2020.0,15.728770,NaN,NaN
36979,Trinidad and Tobago,The Americas,TTO,Fixed broadband subscriptions per 100 inhabitants,2020.0,26.866254,NaN,NaN
36980,United States,The Americas,USA,Fixed broadband subscriptions per 100 inhabitants,2020.0,36.413908,NaN,NaN
36981,Uruguay,The Americas,URY,Fixed broadband subscriptions per 100 inhabitants,2020.0,NaN,NaN,NaN


In [23]:
subpillars[0]
subpillar = subpillars[0]
print(subpillar)

Connectivity Technology


In [24]:
df['higher_is_better'] = True
df['Year'] = 2020
df['Indicator'] = indicator
df['data_col'] = df['Value']
df['Sub-Pillar'] = subpillar

min_rank = df['data_col'].min()
max_rank = df['data_col'].max()

# transform 0-1 rank into 1-6
df['new_rank_score'] = df['data_col'].apply(lambda row: convert_rank(row,old_min=0,old_max=100))

In [25]:
df = df[['Country Name', 'Year','Indicator','data_col','new_rank_score','higher_is_better','Sub-Pillar']]
df

,Country Name,Year,Indicator,data_col,new_rank_score,higher_is_better,Sub-Pillar
36787,Angola,2020,Broadband Density,0.701662,1.035083,True,Connectivity Technology
36788,Benin,2020,Broadband Density,0.247303,1.012365,True,Connectivity Technology
36789,Botswana,2020,Broadband Density,3.057373,1.152869,True,Connectivity Technology
36790,Burkina Faso,2020,Broadband Density,0.066875,1.003344,True,Connectivity Technology
36791,Burundi,2020,Broadband Density,0.035574,1.001779,True,Connectivity Technology
...,...,...,...,...,...,...,...
36978,Suriname,2020,Broadband Density,15.728770,1.786439,True,Connectivity Technology
36979,Trinidad and Tobago,2020,Broadband Density,26.866254,2.343313,True,Connectivity Technology
36980,United States,2020,Broadband Density,36.413908,2.820695,True,Connectivity Technology
36981,Uruguay,2020,Broadband Density,NaN,NaN,True,Connectivity Technology


In [26]:
df.to_csv('../indicator_scores/infrastructure_{}_scores.csv'.format(indicator))

In [27]:
### 3. % of population covered by internet connectivity

In [28]:
indicators[2]

# load data
indicator = indicators[2]
print(indicator)
bf = bnames[bnames['Indicator']==indicator]['Filename'].values[0]
print(bf)

df = pd.read_csv('../../processed/{}.csv'.format(bf))

% of population covered by internet connectivity
ITU_database


In [29]:
df = df[(df.Year == 2019)]
df = df[(df['Indicator name']== 'Individuals using the Internet, total (%)')]
df

,Country,Region,ISO,Indicator name,Year,Value,data_country,data_year
47607,Cabo Verde,Africa,CPV,"Individuals using the Internet, total (%)",2019.0,61.943398,NaN,NaN
47608,Central African Rep.,Africa,CAF,"Individuals using the Internet, total (%)",2019.0,NaN,NaN,NaN
47609,Congo (Rep. of the),Africa,COG,"Individuals using the Internet, total (%)",2019.0,NaN,NaN,NaN
47610,Côte d'Ivoire,Africa,CIV,"Individuals using the Internet, total (%)",2019.0,36.288955,NaN,NaN
47611,Equatorial Guinea,Africa,GNQ,"Individuals using the Internet, total (%)",2019.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
47739,Saint Vincent and the Grenadines,The Americas,VCT,"Individuals using the Internet, total (%)",2019.0,NaN,NaN,NaN
47740,Trinidad and Tobago,The Americas,TTO,"Individuals using the Internet, total (%)",2019.0,NaN,NaN,NaN
47741,United States,The Americas,USA,"Individuals using the Internet, total (%)",2019.0,89.430285,NaN,NaN
47742,Uruguay,The Americas,URY,"Individuals using the Internet, total (%)",2019.0,83.351534,NaN,NaN


In [30]:
subpillars[0]
subpillar = subpillars[0]
print(subpillar)

Connectivity Technology


In [31]:
# create standard columns
# df.rename(columns={'COUNTRY/ECONOMY':'Country Name'}, inplace=True)
df['higher_is_better'] = True
df['Indicator'] = indicator
df['data_col'] = df['Value']
df.rename(columns={'Country':'Country Name'}, inplace=True)
df['Sub-Pillar'] = subpillar

min_rank = df['data_col'].min()
max_rank = df['data_col'].max()

# transform 0-1 rank into 1-6
df['new_rank_score'] = df['data_col'].apply(lambda row: convert_rank(row,old_min=0,old_max=100))

In [32]:
df = df[['Country Name', 'Year','Indicator','data_col','new_rank_score','higher_is_better','Sub-Pillar']]
df

,Country Name,Year,Indicator,data_col,new_rank_score,higher_is_better,Sub-Pillar
47607,Cabo Verde,2019.0,% of population covered by internet connectivity,61.943398,4.097170,True,Connectivity Technology
47608,Central African Rep.,2019.0,% of population covered by internet connectivity,NaN,NaN,True,Connectivity Technology
47609,Congo (Rep. of the),2019.0,% of population covered by internet connectivity,NaN,NaN,True,Connectivity Technology
47610,Côte d'Ivoire,2019.0,% of population covered by internet connectivity,36.288955,2.814448,True,Connectivity Technology
47611,Equatorial Guinea,2019.0,% of population covered by internet connectivity,NaN,NaN,True,Connectivity Technology
...,...,...,...,...,...,...,...
47739,Saint Vincent and the Grenadines,2019.0,% of population covered by internet connectivity,NaN,NaN,True,Connectivity Technology
47740,Trinidad and Tobago,2019.0,% of population covered by internet connectivity,NaN,NaN,True,Connectivity Technology
47741,United States,2019.0,% of population covered by internet connectivity,89.430285,5.471514,True,Connectivity Technology
47742,Uruguay,2019.0,% of population covered by internet connectivity,83.351534,5.167577,True,Connectivity Technology


In [33]:
df.to_csv('../indicator_scores/infrastructure_{}_scores.csv'.format(indicator), index=False)

In [34]:
### 4. % of population covered by mobile 2G

In [35]:
indicators[3]

'% of population covered by mobile 2G+ data connectivity'

In [36]:
# load data
indicator = indicators[3]
print(indicator)
bf = bnames[bnames['Indicator']==indicator]['Filename'].values[0]
print(bf)

df = pd.read_csv('../../processed/{}.csv'.format(bf))

% of population covered by mobile 2G+ data connectivity
countries_mobile_connectivity


In [37]:
df.head(10)

,ISO Code,Country,Region,Year,Cluster,Index,Infrastructure,Affordability,Consumer Readiness,Content and Services,...,Gender gap in mobile ownership,TLDs per capita,E-Government Score,Mobile Social Media Penetration,Apps developed per person,Number of apps in national language,Accessibility of top ranked apps,Cybersecurity Index,data_country,data_year
0,AFG,Afghanistan,South Asia,2014,Discoverer,22.12,21.74,31.79,24.40,14.19,...,0.00,39.55,18.11,3.28,20.98,2.44,4.37,26.50,NaN,NaN
1,AFG,Afghanistan,South Asia,2015,Discoverer,22.99,22.82,30.81,25.28,15.71,...,0.00,39.57,24.27,4.36,22.93,2.79,8.03,25.83,NaN,NaN
2,AFG,Afghanistan,South Asia,2016,Discoverer,23.71,26.92,26.75,26.07,16.83,...,0.00,39.58,30.43,6.73,30.31,2.85,5.90,25.17,NaN,NaN
3,AFG,Afghanistan,South Asia,2017,Discoverer,25.82,33.54,27.22,28.56,17.04,...,0.00,39.47,30.50,7.78,31.62,2.91,6.15,24.50,NaN,NaN
4,AFG,Afghanistan,South Asia,2018,Discoverer,28.39,30.91,42.64,29.24,16.87,...,0.00,39.39,30.56,8.54,36.54,2.96,8.66,17.70,NaN,NaN
5,AFG,Afghanistan,South Asia,2019,Discoverer,28.94,32.34,41.53,29.72,17.58,...,0.00,39.41,41.18,9.39,39.36,3.00,5.72,17.70,NaN,NaN
6,AGO,Angola,Sub-Saharan Africa,2014,Discoverer,32.78,25.99,35.99,44.33,27.85,...,51.65,0.00,29.92,3.99,22.74,53.33,49.09,8.80,NaN,NaN
7,AGO,Angola,Sub-Saharan Africa,2015,Emerging,37.18,33.09,42.01,45.41,30.27,...,57.00,0.31,32.35,5.26,22.12,55.08,58.33,8.47,NaN,NaN
8,AGO,Angola,Sub-Saharan Africa,2016,Emerging,39.85,37.80,44.74,46.47,32.11,...,63.73,0.00,34.78,6.27,27.46,56.52,61.73,8.13,NaN,NaN
9,AGO,Angola,Sub-Saharan Africa,2017,Emerging,42.89,48.60,47.94,46.32,31.36,...,53.97,0.00,37.88,4.26,31.94,57.06,55.09,7.80,NaN,NaN


In [38]:
subpillars[0]
subpillar = subpillars[0]
print(subpillar)

Connectivity Technology


In [39]:
# filter most recent year
df = df[(df.Year==2019)]


In [40]:
# create standard columns
df.rename(columns={'Country':'Country Name'}, inplace=True)
df['higher_is_better'] = True
df['Indicator'] = indicator
df['data_col'] = df['2G Coverage'] 
df['Sub-Pillar'] = subpillar

min_rank = df['data_col'].min()
max_rank = df['data_col'].max()

# transform 0-1 rank into 1-6
df['new_rank_score'] = df['data_col'].apply(lambda row: convert_rank(row,old_min=0,old_max=100))

In [41]:
df = df[['Country Name', 'Year','Indicator','data_col','new_rank_score','higher_is_better','Sub-Pillar']]
df

,Country Name,Year,Indicator,data_col,new_rank_score,higher_is_better,Sub-Pillar
5,Afghanistan,2019,% of population covered by mobile 2G+ data con...,90.00,5.5000,True,Connectivity Technology
11,Angola,2019,% of population covered by mobile 2G+ data con...,90.00,5.5000,True,Connectivity Technology
17,Albania,2019,% of population covered by mobile 2G+ data con...,99.86,5.9930,True,Connectivity Technology
23,United Arab Emirates,2019,% of population covered by mobile 2G+ data con...,100.00,6.0000,True,Connectivity Technology
29,Argentina,2019,% of population covered by mobile 2G+ data con...,98.00,5.9000,True,Connectivity Technology
...,...,...,...,...,...,...,...
995,Samoa,2019,% of population covered by mobile 2G+ data con...,97.00,5.8500,True,Connectivity Technology
1001,Yemen,2019,% of population covered by mobile 2G+ data con...,95.00,5.7500,True,Connectivity Technology
1007,South Africa,2019,% of population covered by mobile 2G+ data con...,99.97,5.9985,True,Connectivity Technology
1013,Zambia,2019,% of population covered by mobile 2G+ data con...,80.40,5.0200,True,Connectivity Technology


In [42]:
df.to_csv('../indicator_scores/infrastructure_{}_scores.csv'.format(indicator), index=False)

In [43]:
### 5. % of population covered by mobile 3G

In [44]:
indicators[4]

# load data
indicator = indicators[4]
print(indicator)
bf = bnames[bnames['Indicator']==indicator]['Filename'].values[0]
print(bf)

df = pd.read_csv('../../processed/{}.csv'.format(bf))

% of population covered by mobile 3G+ data connectivity
countries_mobile_connectivity


In [45]:
df.head(10)

,ISO Code,Country,Region,Year,Cluster,Index,Infrastructure,Affordability,Consumer Readiness,Content and Services,...,Gender gap in mobile ownership,TLDs per capita,E-Government Score,Mobile Social Media Penetration,Apps developed per person,Number of apps in national language,Accessibility of top ranked apps,Cybersecurity Index,data_country,data_year
0,AFG,Afghanistan,South Asia,2014,Discoverer,22.12,21.74,31.79,24.40,14.19,...,0.00,39.55,18.11,3.28,20.98,2.44,4.37,26.50,NaN,NaN
1,AFG,Afghanistan,South Asia,2015,Discoverer,22.99,22.82,30.81,25.28,15.71,...,0.00,39.57,24.27,4.36,22.93,2.79,8.03,25.83,NaN,NaN
2,AFG,Afghanistan,South Asia,2016,Discoverer,23.71,26.92,26.75,26.07,16.83,...,0.00,39.58,30.43,6.73,30.31,2.85,5.90,25.17,NaN,NaN
3,AFG,Afghanistan,South Asia,2017,Discoverer,25.82,33.54,27.22,28.56,17.04,...,0.00,39.47,30.50,7.78,31.62,2.91,6.15,24.50,NaN,NaN
4,AFG,Afghanistan,South Asia,2018,Discoverer,28.39,30.91,42.64,29.24,16.87,...,0.00,39.39,30.56,8.54,36.54,2.96,8.66,17.70,NaN,NaN
5,AFG,Afghanistan,South Asia,2019,Discoverer,28.94,32.34,41.53,29.72,17.58,...,0.00,39.41,41.18,9.39,39.36,3.00,5.72,17.70,NaN,NaN
6,AGO,Angola,Sub-Saharan Africa,2014,Discoverer,32.78,25.99,35.99,44.33,27.85,...,51.65,0.00,29.92,3.99,22.74,53.33,49.09,8.80,NaN,NaN
7,AGO,Angola,Sub-Saharan Africa,2015,Emerging,37.18,33.09,42.01,45.41,30.27,...,57.00,0.31,32.35,5.26,22.12,55.08,58.33,8.47,NaN,NaN
8,AGO,Angola,Sub-Saharan Africa,2016,Emerging,39.85,37.80,44.74,46.47,32.11,...,63.73,0.00,34.78,6.27,27.46,56.52,61.73,8.13,NaN,NaN
9,AGO,Angola,Sub-Saharan Africa,2017,Emerging,42.89,48.60,47.94,46.32,31.36,...,53.97,0.00,37.88,4.26,31.94,57.06,55.09,7.80,NaN,NaN


In [46]:
subpillars[0]
subpillar = subpillars[0]
print(subpillar)

Connectivity Technology


In [47]:
# filter most recent year
df = df[(df.Year==2019)]

# create standard columns
df.rename(columns={'Country':'Country Name'}, inplace=True)
df['higher_is_better'] = True
df['Indicator'] = indicator
df['data_col'] = df['3G Coverage'] 
df['Sub-Pillar'] = subpillar

min_rank = df['data_col'].min()
max_rank = df['data_col'].max()

# transform 0-1 rank into 1-6
df['new_rank_score'] = df['data_col'].apply(lambda row: convert_rank(row,old_min=0,old_max=100))

In [48]:
df = df[['Country Name', 'Year','Indicator','data_col','new_rank_score','higher_is_better','Sub-Pillar']]
df

,Country Name,Year,Indicator,data_col,new_rank_score,higher_is_better,Sub-Pillar
5,Afghanistan,2019,% of population covered by mobile 3G+ data con...,58.7,3.935,True,Connectivity Technology
11,Angola,2019,% of population covered by mobile 3G+ data con...,71.0,4.550,True,Connectivity Technology
17,Albania,2019,% of population covered by mobile 3G+ data con...,97.0,5.850,True,Connectivity Technology
23,United Arab Emirates,2019,% of population covered by mobile 3G+ data con...,100.0,6.000,True,Connectivity Technology
29,Argentina,2019,% of population covered by mobile 3G+ data con...,95.0,5.750,True,Connectivity Technology
...,...,...,...,...,...,...,...
995,Samoa,2019,% of population covered by mobile 3G+ data con...,95.0,5.750,True,Connectivity Technology
1001,Yemen,2019,% of population covered by mobile 3G+ data con...,95.0,5.750,True,Connectivity Technology
1007,South Africa,2019,% of population covered by mobile 3G+ data con...,99.5,5.975,True,Connectivity Technology
1013,Zambia,2019,% of population covered by mobile 3G+ data con...,71.8,4.590,True,Connectivity Technology


In [49]:
df.to_csv('../indicator_scores/infrastructure_{}_scores.csv'.format(indicator), index=False)

In [50]:
### 6. % of population covered by mobile 4G

In [51]:
indicators[5]

# load data
indicator = indicators[5]
print(indicator)
bf = bnames[bnames['Indicator']==indicator]['Filename'].values[0]
print(bf)

df = pd.read_csv('../../processed/{}.csv'.format(bf))

% of population covered by mobile 4G+ data connectivity
countries_mobile_connectivity


In [52]:
df.head(10)

,ISO Code,Country,Region,Year,Cluster,Index,Infrastructure,Affordability,Consumer Readiness,Content and Services,...,Gender gap in mobile ownership,TLDs per capita,E-Government Score,Mobile Social Media Penetration,Apps developed per person,Number of apps in national language,Accessibility of top ranked apps,Cybersecurity Index,data_country,data_year
0,AFG,Afghanistan,South Asia,2014,Discoverer,22.12,21.74,31.79,24.40,14.19,...,0.00,39.55,18.11,3.28,20.98,2.44,4.37,26.50,NaN,NaN
1,AFG,Afghanistan,South Asia,2015,Discoverer,22.99,22.82,30.81,25.28,15.71,...,0.00,39.57,24.27,4.36,22.93,2.79,8.03,25.83,NaN,NaN
2,AFG,Afghanistan,South Asia,2016,Discoverer,23.71,26.92,26.75,26.07,16.83,...,0.00,39.58,30.43,6.73,30.31,2.85,5.90,25.17,NaN,NaN
3,AFG,Afghanistan,South Asia,2017,Discoverer,25.82,33.54,27.22,28.56,17.04,...,0.00,39.47,30.50,7.78,31.62,2.91,6.15,24.50,NaN,NaN
4,AFG,Afghanistan,South Asia,2018,Discoverer,28.39,30.91,42.64,29.24,16.87,...,0.00,39.39,30.56,8.54,36.54,2.96,8.66,17.70,NaN,NaN
5,AFG,Afghanistan,South Asia,2019,Discoverer,28.94,32.34,41.53,29.72,17.58,...,0.00,39.41,41.18,9.39,39.36,3.00,5.72,17.70,NaN,NaN
6,AGO,Angola,Sub-Saharan Africa,2014,Discoverer,32.78,25.99,35.99,44.33,27.85,...,51.65,0.00,29.92,3.99,22.74,53.33,49.09,8.80,NaN,NaN
7,AGO,Angola,Sub-Saharan Africa,2015,Emerging,37.18,33.09,42.01,45.41,30.27,...,57.00,0.31,32.35,5.26,22.12,55.08,58.33,8.47,NaN,NaN
8,AGO,Angola,Sub-Saharan Africa,2016,Emerging,39.85,37.80,44.74,46.47,32.11,...,63.73,0.00,34.78,6.27,27.46,56.52,61.73,8.13,NaN,NaN
9,AGO,Angola,Sub-Saharan Africa,2017,Emerging,42.89,48.60,47.94,46.32,31.36,...,53.97,0.00,37.88,4.26,31.94,57.06,55.09,7.80,NaN,NaN


In [53]:
subpillars[0]
subpillar = subpillars[0]
print(subpillar)

Connectivity Technology


In [54]:
# filter most recent year
df = df[(df.Year==2019)]

# create standard columns
df.rename(columns={'Country':'Country Name'}, inplace=True)
df['higher_is_better'] = True
df['Indicator'] = indicator
df['data_col'] = df['4G Coverage'] 
df['Sub-Pillar'] = subpillar

min_rank = df['data_col'].min()
max_rank = df['data_col'].max()

# transform 0-1 rank into 1-6
df['new_rank_score'] = df['data_col'].apply(lambda row: convert_rank(row,old_min=0,old_max=100))

In [55]:
df = df[['Country Name', 'Year','Indicator','data_col','new_rank_score','higher_is_better','Sub-Pillar']]
df

,Country Name,Year,Indicator,data_col,new_rank_score,higher_is_better,Sub-Pillar
5,Afghanistan,2019,% of population covered by mobile 4G+ data con...,15.00,1.7500,True,Connectivity Technology
11,Angola,2019,% of population covered by mobile 4G+ data con...,50.00,3.5000,True,Connectivity Technology
17,Albania,2019,% of population covered by mobile 4G+ data con...,96.00,5.8000,True,Connectivity Technology
23,United Arab Emirates,2019,% of population covered by mobile 4G+ data con...,99.00,5.9500,True,Connectivity Technology
29,Argentina,2019,% of population covered by mobile 4G+ data con...,89.77,5.4885,True,Connectivity Technology
...,...,...,...,...,...,...,...
995,Samoa,2019,% of population covered by mobile 4G+ data con...,90.00,5.5000,True,Connectivity Technology
1001,Yemen,2019,% of population covered by mobile 4G+ data con...,0.00,1.0000,True,Connectivity Technology
1007,South Africa,2019,% of population covered by mobile 4G+ data con...,95.70,5.7850,True,Connectivity Technology
1013,Zambia,2019,% of population covered by mobile 4G+ data con...,49.10,3.4550,True,Connectivity Technology


In [56]:
df.to_csv('../indicator_scores/infrastructure_{}_scores.csv'.format(indicator), index=False)

In [57]:
### 7. % of population covered by mobile 5G

In [58]:
indicators[6]

# load data
indicator = indicators[6]
print(indicator)
bf = bnames[bnames['Indicator']==indicator]['Filename'].values[0]
print(bf)

df = pd.read_csv('../../processed/{}.csv'.format(bf))

% of population covered by mobile 5G+ data connectivity
countries_mobile_connectivity


In [59]:
df.head(10)

,ISO Code,Country,Region,Year,Cluster,Index,Infrastructure,Affordability,Consumer Readiness,Content and Services,...,Gender gap in mobile ownership,TLDs per capita,E-Government Score,Mobile Social Media Penetration,Apps developed per person,Number of apps in national language,Accessibility of top ranked apps,Cybersecurity Index,data_country,data_year
0,AFG,Afghanistan,South Asia,2014,Discoverer,22.12,21.74,31.79,24.40,14.19,...,0.00,39.55,18.11,3.28,20.98,2.44,4.37,26.50,NaN,NaN
1,AFG,Afghanistan,South Asia,2015,Discoverer,22.99,22.82,30.81,25.28,15.71,...,0.00,39.57,24.27,4.36,22.93,2.79,8.03,25.83,NaN,NaN
2,AFG,Afghanistan,South Asia,2016,Discoverer,23.71,26.92,26.75,26.07,16.83,...,0.00,39.58,30.43,6.73,30.31,2.85,5.90,25.17,NaN,NaN
3,AFG,Afghanistan,South Asia,2017,Discoverer,25.82,33.54,27.22,28.56,17.04,...,0.00,39.47,30.50,7.78,31.62,2.91,6.15,24.50,NaN,NaN
4,AFG,Afghanistan,South Asia,2018,Discoverer,28.39,30.91,42.64,29.24,16.87,...,0.00,39.39,30.56,8.54,36.54,2.96,8.66,17.70,NaN,NaN
5,AFG,Afghanistan,South Asia,2019,Discoverer,28.94,32.34,41.53,29.72,17.58,...,0.00,39.41,41.18,9.39,39.36,3.00,5.72,17.70,NaN,NaN
6,AGO,Angola,Sub-Saharan Africa,2014,Discoverer,32.78,25.99,35.99,44.33,27.85,...,51.65,0.00,29.92,3.99,22.74,53.33,49.09,8.80,NaN,NaN
7,AGO,Angola,Sub-Saharan Africa,2015,Emerging,37.18,33.09,42.01,45.41,30.27,...,57.00,0.31,32.35,5.26,22.12,55.08,58.33,8.47,NaN,NaN
8,AGO,Angola,Sub-Saharan Africa,2016,Emerging,39.85,37.80,44.74,46.47,32.11,...,63.73,0.00,34.78,6.27,27.46,56.52,61.73,8.13,NaN,NaN
9,AGO,Angola,Sub-Saharan Africa,2017,Emerging,42.89,48.60,47.94,46.32,31.36,...,53.97,0.00,37.88,4.26,31.94,57.06,55.09,7.80,NaN,NaN


In [60]:
subpillars[0]
subpillar = subpillars[0]
print(subpillar)

Connectivity Technology


In [61]:
# filter most recent year
df = df[(df.Year==2019)]

# create standard columns
df.rename(columns={'Country':'Country Name'}, inplace=True)
df['higher_is_better'] = True
df['Indicator'] = indicator
df['data_col'] = df['5G Coverage'] 
df['Sub-Pillar'] = subpillar

min_rank = df['data_col'].min()
max_rank = df['data_col'].max()

# transform 0-1 rank into 1-6
df['new_rank_score'] = df['data_col'].apply(lambda row: convert_rank(row,old_min=0,old_max=100))

In [62]:
df = df[['Country Name', 'Year','Indicator','data_col','new_rank_score','higher_is_better','Sub-Pillar']]
df

,Country Name,Year,Indicator,data_col,new_rank_score,higher_is_better,Sub-Pillar
5,Afghanistan,2019,% of population covered by mobile 5G+ data con...,0.0,1.0,True,Connectivity Technology
11,Angola,2019,% of population covered by mobile 5G+ data con...,0.0,1.0,True,Connectivity Technology
17,Albania,2019,% of population covered by mobile 5G+ data con...,0.0,1.0,True,Connectivity Technology
23,United Arab Emirates,2019,% of population covered by mobile 5G+ data con...,100.0,6.0,True,Connectivity Technology
29,Argentina,2019,% of population covered by mobile 5G+ data con...,0.0,1.0,True,Connectivity Technology
...,...,...,...,...,...,...,...
995,Samoa,2019,% of population covered by mobile 5G+ data con...,0.0,1.0,True,Connectivity Technology
1001,Yemen,2019,% of population covered by mobile 5G+ data con...,0.0,1.0,True,Connectivity Technology
1007,South Africa,2019,% of population covered by mobile 5G+ data con...,0.0,1.0,True,Connectivity Technology
1013,Zambia,2019,% of population covered by mobile 5G+ data con...,0.0,1.0,True,Connectivity Technology


In [63]:
df.to_csv('../indicator_scores/infrastructure_{}_scores.csv'.format(indicator), index=False)

In [64]:
### 8. Mobile Coverage Maps

In [65]:
indicators[7]

# load data
indicator = indicators[7]
print(indicator)
bf = bnames[bnames['Indicator']==indicator]['Filename'].values[0]
print(bf)

df = pd.read_csv('../../processed/{}.csv'.format(bf))

Mobile Coverage Maps
ITU_database


In [66]:
df.head(15)

,Country,Region,ISO,Indicator name,Year,Value,data_country,data_year
0,Angola,Africa,AGO,Female mobile phone ownership as a % of total ...,2010.0,NaN,NaN,NaN
1,Benin,Africa,BEN,Female mobile phone ownership as a % of total ...,2010.0,NaN,NaN,NaN
2,Botswana,Africa,BWA,Female mobile phone ownership as a % of total ...,2010.0,NaN,NaN,NaN
3,Burkina Faso,Africa,BFA,Female mobile phone ownership as a % of total ...,2010.0,NaN,NaN,NaN
4,Burundi,Africa,BDI,Female mobile phone ownership as a % of total ...,2010.0,NaN,NaN,NaN
5,Cabo Verde,Africa,CPV,Female mobile phone ownership as a % of total ...,2010.0,NaN,NaN,NaN
6,Cameroon,Africa,CMR,Female mobile phone ownership as a % of total ...,2010.0,NaN,NaN,NaN
7,Central African Rep.,Africa,CAF,Female mobile phone ownership as a % of total ...,2010.0,NaN,NaN,NaN
8,Chad,Africa,TCD,Female mobile phone ownership as a % of total ...,2010.0,NaN,NaN,NaN
9,Congo (Rep. of the),Africa,COG,Female mobile phone ownership as a % of total ...,2010.0,NaN,NaN,NaN


In [67]:
subpillars[0]
subpillar = subpillars[0]
print(subpillar)

Connectivity Technology


In [68]:
df = df[(df.Year == 2020)]
df = df[(df['Indicator name']== 'Mobile-cellular subscriptions per 100 inhabitants')]
df

,Country,Region,ISO,Indicator name,Year,Value,data_country,data_year
32475,Angola,Africa,AGO,Mobile-cellular subscriptions per 100 inhabitants,2020.0,44.559511,NaN,NaN
32476,Benin,Africa,BEN,Mobile-cellular subscriptions per 100 inhabitants,2020.0,91.897280,NaN,NaN
32477,Botswana,Africa,BWA,Mobile-cellular subscriptions per 100 inhabitants,2020.0,162.399011,NaN,NaN
32478,Burkina Faso,Africa,BFA,Mobile-cellular subscriptions per 100 inhabitants,2020.0,105.807440,NaN,NaN
32479,Burundi,Africa,BDI,Mobile-cellular subscriptions per 100 inhabitants,2020.0,55.767172,NaN,NaN
...,...,...,...,...,...,...,...,...
32666,Suriname,The Americas,SUR,Mobile-cellular subscriptions per 100 inhabitants,2020.0,153.305479,NaN,NaN
32667,Trinidad and Tobago,The Americas,TTO,Mobile-cellular subscriptions per 100 inhabitants,2020.0,142.051665,NaN,NaN
32668,United States,The Americas,USA,Mobile-cellular subscriptions per 100 inhabitants,2020.0,NaN,NaN,NaN
32669,Uruguay,The Americas,URY,Mobile-cellular subscriptions per 100 inhabitants,2020.0,NaN,NaN,NaN


In [69]:
# create standard columns
df.rename(columns={'Country':'Country Name'}, inplace=True)
df['higher_is_better'] = True
df['Indicator'] = indicator
df['data_col'] = df['Value'] 
df['Sub-Pillar'] = subpillar

min_rank = df['data_col'].min()
max_rank = df['data_col'].max()

# transform 0-1 rank into 1-6
df['new_rank_score'] = df['data_col'].apply(lambda row: convert_rank(row,old_min=0,old_max=max_rank))

In [70]:
df = df[['Country Name', 'Year','Indicator','data_col','new_rank_score','higher_is_better','Sub-Pillar']]
df

,Country Name,Year,Indicator,data_col,new_rank_score,higher_is_better,Sub-Pillar
32475,Angola,2020.0,Mobile Coverage Maps,44.559511,1.763912,True,Connectivity Technology
32476,Benin,2020.0,Mobile Coverage Maps,91.897280,2.575453,True,Connectivity Technology
32477,Botswana,2020.0,Mobile Coverage Maps,162.399011,3.784108,True,Connectivity Technology
32478,Burkina Faso,2020.0,Mobile Coverage Maps,105.807440,2.813923,True,Connectivity Technology
32479,Burundi,2020.0,Mobile Coverage Maps,55.767172,1.956051,True,Connectivity Technology
...,...,...,...,...,...,...,...
32666,Suriname,2020.0,Mobile Coverage Maps,153.305479,3.628212,True,Connectivity Technology
32667,Trinidad and Tobago,2020.0,Mobile Coverage Maps,142.051665,3.435280,True,Connectivity Technology
32668,United States,2020.0,Mobile Coverage Maps,NaN,NaN,True,Connectivity Technology
32669,Uruguay,2020.0,Mobile Coverage Maps,NaN,NaN,True,Connectivity Technology


In [71]:
df.to_csv('../indicator_scores/infrastructure_{}_scores.csv'.format(indicator), index=False)

In [72]:
### 9. Electricity Density

In [73]:
indicators[8]

# load data
indicator = indicators[8]
print(indicator)
bf = bnames[bnames['Indicator']==indicator]['Filename'].values[0]
print(bf)

df = pd.read_csv('../../processed/{}.csv'.format(bf))

Electricity Density
electricity_density


In [74]:
df.head(20)

,iso_code,country,year,coal_prod_change_pct,coal_prod_change_twh,gas_prod_change_pct,gas_prod_change_twh,oil_prod_change_pct,oil_prod_change_twh,energy_cons_change_pct,...,solar_elec_per_capita,solar_energy_per_capita,gdp,wind_share_elec,wind_cons_change_pct,wind_share_energy,wind_cons_change_twh,wind_consumption,wind_elec_per_capita,wind_energy_per_capita
0,AFG,Afghanistan,1900,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AFG,Afghanistan,1901,NaN,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,AFG,Afghanistan,1902,NaN,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,AFG,Afghanistan,1903,NaN,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,AFG,Afghanistan,1904,NaN,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,AFG,Afghanistan,1905,NaN,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,AFG,Afghanistan,1906,NaN,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,AFG,Afghanistan,1907,NaN,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,AFG,Afghanistan,1908,NaN,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,AFG,Afghanistan,1909,NaN,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [75]:
subpillars[0]
subpillar = subpillars[0]
print(subpillar)

Connectivity Technology


In [76]:
# filter most recent year
df = df[(df.year==2019)]

In [77]:
# create standard columns
df['higher_is_better'] = True
df['Indicator'] = indicator
df['data_col'] = df['per_capita_electricity'] 
df['Sub-Pillar'] = subpillar
df['Country Name'] = df['country']
df['Year'] = df['year']

min_rank = df['data_col'].min()
max_rank = df['data_col'].max()

# transform 0-1 rank into 1-6
df['new_rank_score'] = df['data_col'].apply(lambda row: convert_rank(row,old_min=min_rank,old_max=max_rank))

In [78]:
df = df[['Country Name', 'Year','Indicator','data_col','new_rank_score','higher_is_better','Sub-Pillar']]
df

,Country Name,Year,Indicator,data_col,new_rank_score,higher_is_better,Sub-Pillar
119,Afghanistan,2019,Electricity Density,27.128,1.002387,True,Connectivity Technology
239,Africa,2019,Electricity Density,665.189,1.058527,True,Connectivity Technology
359,Albania,2019,Electricity Density,1796.578,1.158072,True,Connectivity Technology
479,Algeria,2019,Electricity Density,1726.900,1.151941,True,Connectivity Technology
519,American Samoa,2019,Electricity Density,2963.636,1.260755,True,Connectivity Technology
...,...,...,...,...,...,...,...
16903,Western Sahara,2019,Electricity Density,0.000,1.000000,True,Connectivity Technology
17023,World,2019,Electricity Density,3357.738,1.295430,True,Connectivity Technology
17074,Yemen,2019,Electricity Density,86.190,1.007583,True,Connectivity Technology
17311,Zambia,2019,Electricity Density,943.655,1.083027,True,Connectivity Technology


In [79]:
df.to_csv('../indicator_scores/infrastructure_{}_scores.csv'.format(indicator), index=False)

In [80]:
### 10. % of population covered by electricity

In [81]:
indicators[9]

# load data
indicator = indicators[9]
print(indicator)
bf = bnames[bnames['Indicator']==indicator]['Filename'].values[0]
print(bf)

df = pd.read_csv('../../processed/{}.csv'.format(bf))

% of population covered by electricity
population_electricity_coverage


In [82]:
df.head(15)

,1960,1961,1962,1963,1964,1965,1966,1967,1968,1969,...,2017,2018,2019,2020,Country Name,Country Code,Indicator Name,Indicator Code,data_country,data_year
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,100.000000,100.000000,100.000000,NaN,Aruba,ABW,Access to electricity (% of population),EG.ELC.ACCS.ZS,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,39.754201,42.168241,43.640661,NaN,Africa Eastern and Southern,AFE,Access to electricity (% of population),EG.ELC.ACCS.ZS,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,97.700000,98.715622,97.700000,NaN,Afghanistan,AFG,Access to electricity (% of population),EG.ELC.ACCS.ZS,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,48.848205,51.253253,51.341421,NaN,Africa Western and Central,AFW,Access to electricity (% of population),EG.ELC.ACCS.ZS,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,43.001610,45.290000,45.670315,NaN,Angola,AGO,Access to electricity (% of population),EG.ELC.ACCS.ZS,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,99.890000,100.000000,100.000000,NaN,Albania,ALB,Access to electricity (% of population),EG.ELC.ACCS.ZS,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,100.000000,100.000000,100.000000,NaN,Andorra,AND,Access to electricity (% of population),EG.ELC.ACCS.ZS,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,89.623427,88.306324,89.512282,NaN,Arab World,ARB,Access to electricity (% of population),EG.ELC.ACCS.ZS,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,100.000000,100.000000,100.000000,NaN,United Arab Emirates,ARE,Access to electricity (% of population),EG.ELC.ACCS.ZS,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,100.000000,100.000000,100.000000,NaN,Argentina,ARG,Access to electricity (% of population),EG.ELC.ACCS.ZS,NaN,NaN


In [83]:
subpillars[0]
subpillar = subpillars[0]
print(subpillar)

Connectivity Technology


In [84]:
# create standard columns
df['higher_is_better'] = True
df['Indicator'] = indicator
df['data_col'] = df['2019'] 
df['Year'] = 2019
df['Sub-Pillar'] = subpillar

min_rank = df['data_col'].min()
max_rank = df['data_col'].max()

# transform 0-1 rank into 1-6
df['new_rank_score'] = df['data_col'].apply(lambda row: convert_rank(row,old_min=0,old_max=100))

In [85]:
df = df[['Country Name', 'Year','Indicator','data_col','new_rank_score','higher_is_better','Sub-Pillar']]
df


,Country Name,Year,Indicator,data_col,new_rank_score,higher_is_better,Sub-Pillar
0,Aruba,2019,% of population covered by electricity,100.000000,6.000000,True,Connectivity Technology
1,Africa Eastern and Southern,2019,% of population covered by electricity,43.640661,3.182033,True,Connectivity Technology
2,Afghanistan,2019,% of population covered by electricity,97.700000,5.885000,True,Connectivity Technology
3,Africa Western and Central,2019,% of population covered by electricity,51.341421,3.567071,True,Connectivity Technology
4,Angola,2019,% of population covered by electricity,45.670315,3.283516,True,Connectivity Technology
...,...,...,...,...,...,...,...
261,Kosovo,2019,% of population covered by electricity,100.000000,6.000000,True,Connectivity Technology
262,"Yemen, Rep.",2019,% of population covered by electricity,72.751701,4.637585,True,Connectivity Technology
263,South Africa,2019,% of population covered by electricity,85.000000,5.250000,True,Connectivity Technology
264,Zambia,2019,% of population covered by electricity,43.000000,3.150000,True,Connectivity Technology


In [86]:
df.to_csv('../indicator_scores/infrastructure_{}_scores.csv'.format(indicator), index=False)

In [87]:
### 11. Electricity Supply Quality

In [88]:
indicators[10]

# load data
indicator = indicators[10]
print(indicator)
bf = bnames[bnames['Indicator']==indicator]['Filename'].values[0]
print(bf)

df = pd.read_csv('../../processed/{}.csv'.format(bf))

Electricity supply quality
TCdata360


In [89]:
df.head(10)

,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,...,2008-2009,2009-2010,2010-2011,2011-2012,2012-2013,2013-2014,2014-2015,2015-2016,2016-2017,2017-2018
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,40.900000,54.127067,NaN,40.920500,38.903010,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,66.000000,49.000000,NaN,87.000000,92.000000,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,3.858249,3.165548,NaN,2.823529,2.601164,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,127.000000,142.000000,NaN,148.000000,144.000000,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,2.797549,2.242699,NaN,2.028571,2.177586,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,130.000000,142.000000,NaN,148.000000,144.000000,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,2.196592,2.090002,NaN,2.030303,2.249658,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,138.000000,141.000000,NaN,148.000000,141.000000,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,2.793415,2.468175,NaN,2.352941,2.260678,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,115.000000,127.000000,NaN,139.000000,138.000000,NaN,NaN,NaN


In [90]:
subpillars[0]
subpillar = subpillars[0]
print(subpillar)

Connectivity Technology


In [91]:
df = df[(df.Indicator == 'Quality of electricity supply')]
df = df[(df['Subindicator Type'] == '1-7 Best')]
df

,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,...,2008-2009,2009-2010,2010-2011,2011-2012,2012-2013,2013-2014,2014-2015,2015-2016,2016-2017,2017-2018
18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,1.532095,1.418499,NaN,1.656250,1.714501,NaN,NaN,NaN
304,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.804718,2.711991,3.996428,4.984440,4.830509,4.452814,4.672173,4.724386,4.450146,4.607356
588,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,6.409734,6.438373,6.508670,6.410395,6.394012,6.457188,6.602285,6.602285,6.695661,6.519955
872,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.253992,3.482148,3.830355,3.757777,3.480835,3.057727,2.626327,2.560346,2.702864,3.007207
1158,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.974285,4.222707,4.447690,4.765749,4.902742,5.207751,5.077727,4.768738,4.783776,4.780178
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41754,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.220279,3.322920,3.638985,3.280374,3.143945,3.961566,4.189500,4.112941,4.366921,4.338947
42038,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1.537728,1.403162,1.454153,1.450235,NaN,1.215040,1.189313
42322,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.363019,3.468128,3.784828,3.719002,3.904491,3.768681,3.578428,2.870723,3.006041,3.920007
42608,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.928674,3.047631,3.267132,3.482606,3.518804,3.103123,3.326362,3.346282,2.506594,2.520937


In [92]:
df['higher_is_better'] = True
df['Year'] = 2018
df['Indicator'] = indicator
df['data_col'] = df['2017-2018']
df['Sub-Pillar'] = subpillar

min_rank = df['data_col'].min()
max_rank = df['data_col'].max()

# transform 0-1 rank into 1-6
df['new_rank_score'] = df['data_col'].apply(lambda row: convert_rank(row,old_min=0,old_max=100))

In [93]:
df = df[['Country Name', 'Year','Indicator','data_col','new_rank_score','higher_is_better','Sub-Pillar']]
df

,Country Name,Year,Indicator,data_col,new_rank_score,higher_is_better,Sub-Pillar
18,Angola,2018,Electricity supply quality,NaN,NaN,True,Connectivity Technology
304,Albania,2018,Electricity supply quality,4.607356,1.230368,True,Connectivity Technology
588,United Arab Emirates,2018,Electricity supply quality,6.519955,1.325998,True,Connectivity Technology
872,Argentina,2018,Electricity supply quality,3.007207,1.150360,True,Connectivity Technology
1158,Armenia,2018,Electricity supply quality,4.780178,1.239009,True,Connectivity Technology
...,...,...,...,...,...,...,...
41754,Vietnam,2018,Electricity supply quality,4.338947,1.216947,True,Connectivity Technology
42038,"Yemen, Rep.",2018,Electricity supply quality,1.189313,1.059466,True,Connectivity Technology
42322,South Africa,2018,Electricity supply quality,3.920007,1.196000,True,Connectivity Technology
42608,Zambia,2018,Electricity supply quality,2.520937,1.126047,True,Connectivity Technology


In [94]:
df.to_csv('../indicator_scores/infrastructure_{}_scores.csv'.format(indicator), index=False)

In [95]:
### 12. Electricity Environmental Sustainability

In [96]:
indicators[11]

# load data
indicator = indicators[11]
print(indicator)
bf = bnames[bnames['Indicator']==indicator]['Filename'].values[0]
print(bf)

df = pd.read_csv('../../processed/{}.csv'.format(bf))

Electricity Environmental Sustainability
sustainability_index


In [97]:
df.head(15)

,Country Code ISO3,Country,2021 SDG Index Score,2021 SDG Index Rank,Percentage missing values,Spillover Score (0-100),Regional Score (0-100),Regions used for the SDR,Population in 2020,Goal 1 Dash,...,Goal 10 Regional Score,Goal 11 Regional Score,Goal 12 Regional Score,Goal 13 Regional Score,Goal 14 Regional Score,Goal 15 Regional Score,Goal 16 Regional Score,Goal 17 Regional Score,data_country,data_year
0,AFG,Afghanistan,53.9,137.0,9.3,99.3,71.4,E. Europe & C. Asia,"38,928,341",grey,...,72.362762,76.443228,77.629345,83.041913,58.865821,69.499564,72.403938,63.386460,NaN,NaN
1,AGO,Angola,50.3,154.0,0.0,97.0,51.9,Africa,"32,866,268",red,...,37.334205,52.728915,88.024702,96.266527,68.010492,66.835003,56.050983,51.652955,NaN,NaN
2,ALB,Albania,71.0,64.0,3.3,94.3,71.4,E. Europe & C. Asia,"2,877,800",yellow,...,72.362762,76.443228,77.629345,83.041913,58.865821,69.499564,72.403938,63.386460,NaN,NaN
3,AND,Andorra,NaN,NaN,51.2,NaN,71.4,E. Europe & C. Asia,"77,265",grey,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ARE,United Arab Emirates,70.2,71.0,11.0,38.8,67.1,MENA,"9,890,400",green,...,66.178091,58.928271,76.842146,71.475510,62.776615,57.356212,69.093831,56.231255,NaN,NaN
5,ARG,Argentina,72.8,52.0,1.1,94.5,68.6,LAC,"45,195,777",yellow,...,27.901676,77.087594,76.727283,85.373283,62.781114,60.904217,60.888934,64.295156,NaN,NaN
6,ARM,Armenia,71.8,58.0,3.5,96.7,71.4,E. Europe & C. Asia,"2,963,234",yellow,...,72.362762,76.443228,77.629345,83.041913,58.865821,69.499564,72.403938,63.386460,NaN,NaN
7,ATG,Antigua and Barbuda,NaN,NaN,30.8,NaN,68.6,LAC,"97,928",grey,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,AUS,Australia,75.6,35.0,1.1,63.9,77.2,OECD,"25,499,881",yellow,...,74.475203,88.911367,58.055279,64.112721,63.360546,74.102776,82.316815,65.773824,NaN,NaN
9,AUT,Austria,82.1,6.0,3.5,59.5,77.2,OECD,"9,006,400",green,...,74.475203,88.911367,58.055279,64.112721,63.360546,74.102776,82.316815,65.773824,NaN,NaN


In [98]:
subpillars[0]
subpillar = subpillars[0]
print(subpillar)

Connectivity Technology


In [99]:
# create standard columns
df.rename(columns={'Country':'Country Name'}, inplace=True)
df['higher_is_better'] = True
df['Indicator'] = indicator
df['data_col'] = df['2021 SDG Index Score'] 
df['Year'] = 2021
df['Sub-Pillar'] = subpillar

min_rank = df['data_col'].min()
max_rank = df['data_col'].max()

# transform 0-1 rank into 1-6
df['new_rank_score'] = df['data_col'].apply(lambda row: convert_rank(row,old_min=0,old_max=100))

In [100]:
df = df[['Country Name', 'Year','Indicator','data_col','new_rank_score','higher_is_better','Sub-Pillar']]
df

,Country Name,Year,Indicator,data_col,new_rank_score,higher_is_better,Sub-Pillar
0,Afghanistan,2021,Electricity Environmental Sustainability,53.9,3.695,True,Connectivity Technology
1,Angola,2021,Electricity Environmental Sustainability,50.3,3.515,True,Connectivity Technology
2,Albania,2021,Electricity Environmental Sustainability,71.0,4.550,True,Connectivity Technology
3,Andorra,2021,Electricity Environmental Sustainability,NaN,NaN,True,Connectivity Technology
4,United Arab Emirates,2021,Electricity Environmental Sustainability,70.2,4.510,True,Connectivity Technology
...,...,...,...,...,...,...,...
200,Sub-Saharan Africa,2021,Electricity Environmental Sustainability,51.9,3.595,True,Connectivity Technology
201,Low-income Countries,2021,Electricity Environmental Sustainability,51.0,3.550,True,Connectivity Technology
202,Lower-middle-income Countries,2021,Electricity Environmental Sustainability,60.1,4.005,True,Connectivity Technology
203,Upper-middle-income Countries,2021,Electricity Environmental Sustainability,70.8,4.540,True,Connectivity Technology


In [101]:
df.to_csv('../indicator_scores/infrastructure_{}_scores.csv'.format(indicator), index=False)

In [102]:
### 13. Cellphone Signal Density

In [103]:
indicators[12]

# load data
indicator = indicators[12]
print(indicator)
bf = bnames[bnames['Indicator']==indicator]['Filename'].values[0]
print(bf)

df = pd.read_csv('../../processed/{}.csv'.format(bf))

Cellphone Signal Density
mobile_density


In [104]:
df.head(10)

,Unnamed: 0,ISO Code,Country,Region,Year,Network coverage
0,1,AFG,Afghanistan,South Asia,2014,28.919998
1,2,AFG,Afghanistan,South Asia,2015,31.799999
2,3,AFG,Afghanistan,South Asia,2016,33.040001
3,4,AFG,Afghanistan,South Asia,2017,36.340000
4,5,AFG,Afghanistan,South Asia,2018,30.931999
5,6,AFG,Afghanistan,South Asia,2019,38.480000
6,7,AGO,Angola,Sub-Saharan Africa,2014,51.445770
7,8,AGO,Angola,Sub-Saharan Africa,2015,53.906952
8,9,AGO,Angola,Sub-Saharan Africa,2016,61.174950
9,10,AGO,Angola,Sub-Saharan Africa,2017,60.800003


In [105]:
subpillars[0]
subpillar = subpillars[0]
print(subpillar)

Connectivity Technology


In [106]:
df = df[(df.Year == 2019)]

# create standard columns
df.rename(columns={'Country':'Country Name'}, inplace=True)
df['higher_is_better'] = True
df['Indicator'] = indicator
df['data_col'] = df['Network coverage'] 
df['Sub-Pillar'] = subpillar

min_rank = df['data_col'].min()
max_rank = df['data_col'].max()

# transform 0-1 rank into 1-6
df['new_rank_score'] = df['data_col'].apply(lambda row: convert_rank(row,old_min=0,old_max=100))

In [107]:
df = df[['Country Name', 'Year','Indicator','data_col','new_rank_score','higher_is_better','Sub-Pillar']]
df

,Country Name,Year,Indicator,data_col,new_rank_score,higher_is_better,Sub-Pillar
5,Afghanistan,2019,Cellphone Signal Density,38.480000,2.92400,True,Connectivity Technology
11,Angola,2019,Cellphone Signal Density,57.400002,3.87000,True,Connectivity Technology
17,Albania,2019,Cellphone Signal Density,87.186005,5.35930,True,Connectivity Technology
23,United Arab Emirates,2019,Cellphone Signal Density,99.599998,5.98000,True,Connectivity Technology
29,Argentina,2019,Cellphone Signal Density,83.708000,5.18540,True,Connectivity Technology
...,...,...,...,...,...,...,...
995,Samoa,2019,Cellphone Signal Density,83.699997,5.18500,True,Connectivity Technology
1001,Yemen,2019,Cellphone Signal Density,47.500000,3.37500,True,Connectivity Technology
1007,South Africa,2019,Cellphone Signal Density,88.077003,5.40385,True,Connectivity Technology
1013,Zambia,2019,Cellphone Signal Density,56.400002,3.82000,True,Connectivity Technology


In [108]:
df.to_csv('../indicator_scores/infrastructure_{}_scores.csv'.format(indicator), index=False)

In [109]:
### 14. Telecommunication Infrastructure Index (TII)

In [110]:
indicators[13]

# load data
indicator = indicators[13]
print(indicator)
bf = bnames[bnames['Indicator']==indicator]['Filename'].values[0]
print(bf)

df = pd.read_csv('../../processed/{}.csv'.format(bf))

Telecommunication Infrastructure Index (TII)
e_government_index


In [111]:
df.head(10)

,Survey Year,Country Name,E-Government Rank,E-Government Index,E-Participation Index,Online Service Index,Human Capital Index,Telecommunication Infrastructure Index
0,2020,Iraq,143,0.4360,0.3095,0.3353,0.4358,0.5370
1,2020,Ireland,27,0.8433,0.8571,0.7706,0.9494,0.8100
2,2020,Israel,30,0.8361,0.7143,0.7471,0.8924,0.8689
3,2020,Italy,37,0.8231,0.8214,0.8294,0.8466,0.7932
4,2020,Jamaica,114,0.5392,0.3690,0.3882,0.7142,0.5151
5,2020,Japan,14,0.8989,0.9881,0.9059,0.8684,0.9223
6,2020,Jordan,117,0.5309,0.3333,0.3588,0.6800,0.5540
7,2020,Kazakhstan,29,0.8375,0.8810,0.9235,0.8866,0.7024
8,2020,Kenya,116,0.5326,0.5952,0.6765,0.5812,0.3402
9,2020,Kiribati,145,0.4320,0.5595,0.4941,0.6778,0.1241


In [112]:
subpillars[0]
subpillar = subpillars[0]
print(subpillar)

Connectivity Technology


In [113]:
# create standard columns
# df.rename(columns={'COUNTRY/ECONOMY':'Country Name'}, inplace=True)

df['higher_is_better'] = True
df['Indicator'] = indicator
df['data_col'] = df['Telecommunication Infrastructure Index'] 
df['Year'] = df['Survey Year']
df['Sub-Pillar'] = subpillar

min_rank = df['data_col'].min()
max_rank = df['data_col'].max()

# transform 0-1 rank into 1-6
df['new_rank_score'] = df['data_col'].apply(lambda row: convert_rank_b(row,old_min=0,old_max=1))


In [114]:
df = df[['Country Name', 'Year','Indicator','data_col','new_rank_score','higher_is_better','Sub-Pillar']]
df

,Country Name,Year,Indicator,data_col,new_rank_score,higher_is_better,Sub-Pillar
0,Iraq,2020,Telecommunication Infrastructure Index (TII),0.5370,3.6850,True,Connectivity Technology
1,Ireland,2020,Telecommunication Infrastructure Index (TII),0.8100,5.0500,True,Connectivity Technology
2,Israel,2020,Telecommunication Infrastructure Index (TII),0.8689,5.3445,True,Connectivity Technology
3,Italy,2020,Telecommunication Infrastructure Index (TII),0.7932,4.9660,True,Connectivity Technology
4,Jamaica,2020,Telecommunication Infrastructure Index (TII),0.5151,3.5755,True,Connectivity Technology
...,...,...,...,...,...,...,...
188,Senegal,2020,Telecommunication Infrastructure Index (TII),0.4358,3.1790,True,Connectivity Technology
189,Serbia,2020,Telecommunication Infrastructure Index (TII),0.6200,4.1000,True,Connectivity Technology
190,Seychelles,2020,Telecommunication Infrastructure Index (TII),0.6925,4.4625,True,Connectivity Technology
191,Singapore,2020,Telecommunication Infrastructure Index (TII),0.8899,5.4495,True,Connectivity Technology


In [115]:
df.to_csv('../indicator_scores/infrastructure_{}_scores.csv'.format(indicator), index=False)

In [116]:
### 15. GSMA Mobile Connectivity Index

In [117]:
indicators[14]

# load data
indicator = indicators[14]
print(indicator)
bf = bnames[bnames['Indicator']==indicator]['Filename'].values[0]
print(bf)

df = pd.read_csv('../../processed/{}.csv'.format(bf))

GSMA Mobile Connectivity Index 
countries_mobile_connectivity


In [118]:
df.head(10)

,ISO Code,Country,Region,Year,Cluster,Index,Infrastructure,Affordability,Consumer Readiness,Content and Services,...,Gender gap in mobile ownership,TLDs per capita,E-Government Score,Mobile Social Media Penetration,Apps developed per person,Number of apps in national language,Accessibility of top ranked apps,Cybersecurity Index,data_country,data_year
0,AFG,Afghanistan,South Asia,2014,Discoverer,22.12,21.74,31.79,24.40,14.19,...,0.00,39.55,18.11,3.28,20.98,2.44,4.37,26.50,NaN,NaN
1,AFG,Afghanistan,South Asia,2015,Discoverer,22.99,22.82,30.81,25.28,15.71,...,0.00,39.57,24.27,4.36,22.93,2.79,8.03,25.83,NaN,NaN
2,AFG,Afghanistan,South Asia,2016,Discoverer,23.71,26.92,26.75,26.07,16.83,...,0.00,39.58,30.43,6.73,30.31,2.85,5.90,25.17,NaN,NaN
3,AFG,Afghanistan,South Asia,2017,Discoverer,25.82,33.54,27.22,28.56,17.04,...,0.00,39.47,30.50,7.78,31.62,2.91,6.15,24.50,NaN,NaN
4,AFG,Afghanistan,South Asia,2018,Discoverer,28.39,30.91,42.64,29.24,16.87,...,0.00,39.39,30.56,8.54,36.54,2.96,8.66,17.70,NaN,NaN
5,AFG,Afghanistan,South Asia,2019,Discoverer,28.94,32.34,41.53,29.72,17.58,...,0.00,39.41,41.18,9.39,39.36,3.00,5.72,17.70,NaN,NaN
6,AGO,Angola,Sub-Saharan Africa,2014,Discoverer,32.78,25.99,35.99,44.33,27.85,...,51.65,0.00,29.92,3.99,22.74,53.33,49.09,8.80,NaN,NaN
7,AGO,Angola,Sub-Saharan Africa,2015,Emerging,37.18,33.09,42.01,45.41,30.27,...,57.00,0.31,32.35,5.26,22.12,55.08,58.33,8.47,NaN,NaN
8,AGO,Angola,Sub-Saharan Africa,2016,Emerging,39.85,37.80,44.74,46.47,32.11,...,63.73,0.00,34.78,6.27,27.46,56.52,61.73,8.13,NaN,NaN
9,AGO,Angola,Sub-Saharan Africa,2017,Emerging,42.89,48.60,47.94,46.32,31.36,...,53.97,0.00,37.88,4.26,31.94,57.06,55.09,7.80,NaN,NaN


In [119]:
subpillars[0]
subpillar = subpillars[0]
print(subpillar)

Connectivity Technology


In [120]:
# filter most recent year
df = df[(df.Year==2019)]

# create standard columns
df.rename(columns={'Country':'Country Name'}, inplace=True)

df['higher_is_better'] = True
df['Indicator'] = indicator
df['data_col'] = df['Index'] 
df['Sub-Pillar'] = subpillar

min_rank = df['data_col'].min()
max_rank = df['data_col'].max()

# transform 0-1 rank into 1-6
df['new_rank_score'] = df['data_col'].apply(lambda row: convert_rank(row,old_min=0,old_max=100))

In [121]:
df = df[['Country Name', 'Year','Indicator','data_col','new_rank_score','higher_is_better','Sub-Pillar']]
df

,Country Name,Year,Indicator,data_col,new_rank_score,higher_is_better,Sub-Pillar
5,Afghanistan,2019,GSMA Mobile Connectivity Index,28.94,2.4470,True,Connectivity Technology
11,Angola,2019,GSMA Mobile Connectivity Index,43.53,3.1765,True,Connectivity Technology
17,Albania,2019,GSMA Mobile Connectivity Index,67.89,4.3945,True,Connectivity Technology
23,United Arab Emirates,2019,GSMA Mobile Connectivity Index,78.23,4.9115,True,Connectivity Technology
29,Argentina,2019,GSMA Mobile Connectivity Index,67.16,4.3580,True,Connectivity Technology
...,...,...,...,...,...,...,...
995,Samoa,2019,GSMA Mobile Connectivity Index,60.20,4.0100,True,Connectivity Technology
1001,Yemen,2019,GSMA Mobile Connectivity Index,29.71,2.4855,True,Connectivity Technology
1007,South Africa,2019,GSMA Mobile Connectivity Index,60.14,4.0070,True,Connectivity Technology
1013,Zambia,2019,GSMA Mobile Connectivity Index,35.34,2.7670,True,Connectivity Technology


In [122]:
df.to_csv('../indicator_scores/infrastructure_{}_scores.csv'.format(indicator), index=False)

In [123]:
### 16. Spectrum Allocated to Mobile Providers

In [124]:
indicators[15]

# load data
indicator = indicators[15]
print(indicator)
bf = bnames[bnames['Indicator']==indicator]['Filename'].values[0]
print(bf)

df = pd.read_csv('../../processed/{}.csv'.format(bf))

Spectrum Allocated to Mobile Providers
spectrum_allocated_mobile_providers


In [125]:
df.head(15)

,ISO Code,Country,Region,Year,Cluster,Index,Infrastructure,Affordability,Consumer Readiness,Content and Services,...,Gender gap in mobile ownership,TLDs per capita,E-Government Score,Mobile Social Media Penetration,Apps developed per person,Number of apps in national language,Accessibility of top ranked apps,Cybersecurity Index,data_country,data_year
0,AFG,Afghanistan,South Asia,2014,Discoverer,22.12,21.74,31.79,24.40,14.19,...,0.00,39.55,18.11,3.28,20.98,2.44,4.37,26.50,NaN,NaN
1,AFG,Afghanistan,South Asia,2015,Discoverer,22.99,22.82,30.81,25.28,15.71,...,0.00,39.57,24.27,4.36,22.93,2.79,8.03,25.83,NaN,NaN
2,AFG,Afghanistan,South Asia,2016,Discoverer,23.71,26.92,26.75,26.07,16.83,...,0.00,39.58,30.43,6.73,30.31,2.85,5.90,25.17,NaN,NaN
3,AFG,Afghanistan,South Asia,2017,Discoverer,25.82,33.54,27.22,28.56,17.04,...,0.00,39.47,30.50,7.78,31.62,2.91,6.15,24.50,NaN,NaN
4,AFG,Afghanistan,South Asia,2018,Discoverer,28.39,30.91,42.64,29.24,16.87,...,0.00,39.39,30.56,8.54,36.54,2.96,8.66,17.70,NaN,NaN
5,AFG,Afghanistan,South Asia,2019,Discoverer,28.94,32.34,41.53,29.72,17.58,...,0.00,39.41,41.18,9.39,39.36,3.00,5.72,17.70,NaN,NaN
6,AGO,Angola,Sub-Saharan Africa,2014,Discoverer,32.78,25.99,35.99,44.33,27.85,...,51.65,0.00,29.92,3.99,22.74,53.33,49.09,8.80,NaN,NaN
7,AGO,Angola,Sub-Saharan Africa,2015,Emerging,37.18,33.09,42.01,45.41,30.27,...,57.00,0.31,32.35,5.26,22.12,55.08,58.33,8.47,NaN,NaN
8,AGO,Angola,Sub-Saharan Africa,2016,Emerging,39.85,37.80,44.74,46.47,32.11,...,63.73,0.00,34.78,6.27,27.46,56.52,61.73,8.13,NaN,NaN
9,AGO,Angola,Sub-Saharan Africa,2017,Emerging,42.89,48.60,47.94,46.32,31.36,...,53.97,0.00,37.88,4.26,31.94,57.06,55.09,7.80,NaN,NaN


In [126]:
subpillars[0]
subpillar = subpillars[0]
print(subpillar)

Connectivity Technology


In [127]:
# filter most recent year
df = df[(df.Year==2019)]

# create standard columns
df.rename(columns={'Country':'Country Name'}, inplace=True)

df['higher_is_better'] = True
df['Indicator'] = indicator
df['data_col'] = df['Spectrum'] 
df['Sub-Pillar'] = subpillar

min_rank = df['data_col'].min()
max_rank = df['data_col'].max()

# transform 0-1 rank into 1-6
df['new_rank_score'] = df['data_col'].apply(lambda row: convert_rank(row,old_min=0,old_max=100))

In [128]:
df = df[['Country Name', 'Year','Indicator','data_col','new_rank_score','higher_is_better','Sub-Pillar']]
df

,Country Name,Year,Indicator,data_col,new_rank_score,higher_is_better,Sub-Pillar
5,Afghanistan,2019,Spectrum Allocated to Mobile Providers,17.62,1.8810,True,Connectivity Technology
11,Angola,2019,Spectrum Allocated to Mobile Providers,48.22,3.4110,True,Connectivity Technology
17,Albania,2019,Spectrum Allocated to Mobile Providers,52.85,3.6425,True,Connectivity Technology
23,United Arab Emirates,2019,Spectrum Allocated to Mobile Providers,82.44,5.1220,True,Connectivity Technology
29,Argentina,2019,Spectrum Allocated to Mobile Providers,63.38,4.1690,True,Connectivity Technology
...,...,...,...,...,...,...,...
995,Samoa,2019,Spectrum Allocated to Mobile Providers,30.78,2.5390,True,Connectivity Technology
1001,Yemen,2019,Spectrum Allocated to Mobile Providers,18.55,1.9275,True,Connectivity Technology
1007,South Africa,2019,Spectrum Allocated to Mobile Providers,30.66,2.5330,True,Connectivity Technology
1013,Zambia,2019,Spectrum Allocated to Mobile Providers,23.28,2.1640,True,Connectivity Technology


In [129]:
df.to_csv('../indicator_scores/infrastructure_{}_scores.csv'.format(indicator), index=False)

In [130]:
### 17. Internet Exchange Points (IXPs) 

In [131]:
indicators[16]

# load data
indicator = indicators[16]
print(indicator)
bf = bnames[bnames['Indicator']==indicator]['Filename'].values[0]
print(bf)

df = pd.read_csv('../../processed/{}.csv'.format(bf))



Internet Exchange Points (IXPs) map
countries_ixp


In [132]:
df['Country Name'] = df['Country']
df.head(15)

,Region,Country,City,IXP Name,Participants,Peak,Avg,IPv6,Prefixes,Established,URL,Country Name
0,Asia-Pacific,Afghanistan,Kabul,National Internet Exchange of Afghanistan,20,6.4G,2.19G,NaN,NaN,18-Jul,NaN,Afghanistan
1,Europe,Albania,Tirana,Albanian Neutral Internet eXchange,16,2.42G,35M,NaN,NaN,2018,NaN,Albania
2,NaN,Albania,Tirana,Albania IX,0,NaN,NaN,NaN,NaN,2011,NaN,Albania
3,Africa,Algeria,Algiers,Algeria Internet Exchange,0,NaN,NaN,NaN,NaN,NaN,NaN,Algeria
4,NaN,Angola,Luanda,Angola Internet Exchange,21,1.6G,NaN,NaN,512.0,17-Mar-06,NaN,Angola
5,NaN,Angola,Luanda,Ponto de Intercambio Internet Angola,0,NaN,NaN,NaN,NaN,NaN,NaN,Angola
6,NaN,Angola,Luanda,ANGONIX,21,16G,NaN,NaN,5495.0,9-Mar-15,NaN,Angola
7,Latin America,Argentina,Bahía Blanca,CABASE IXP Bahía Blanca,15,713M,NaN,NaN,NaN,27-Apr-13,NaN,Argentina
8,NaN,Argentina,Bariloche,CABASE IXP Bariloche,8,NaN,NaN,NaN,NaN,15-Sep-14,NaN,Argentina
9,NaN,Argentina,Buenos Aires,CABASE IXP Buenos Aires,125,9.19G,NaN,NaN,17224.0,1-Apr-98,NaN,Argentina


In [133]:
subpillars[0]
subpillar = subpillars[0]
print(subpillar)

Connectivity Technology


In [134]:
agg_df = df.groupby('Country Name').agg({'Country':'count'})
agg_df

,Country
Country Name,
Afghanistan,1
Albania,2
Algeria,1
Angola,3
Argentina,33
...,...
Uzbekistan,3
Vanuatu,1
Viet Nam,3


In [135]:
# create standard columns
# df.rename(columns={'COUNTRY/ECONOMY':'Country Name'}, inplace=True)

agg_df['higher_is_better'] = True
agg_df['Indicator'] = indicator
agg_df['data_col'] = agg_df['Country'] 
agg_df['Year'] = 2019
agg_df['Sub-Pillar'] = subpillar

min_rank = agg_df['data_col'].min()
max_rank = agg_df['data_col'].max()

# transform 0-1 rank into 1-6
agg_df['new_rank_score'] = agg_df['data_col'].apply(lambda row: convert_rank(row,old_min=min_rank,old_max=max_rank))

In [136]:
agg_df = agg_df[['Year','Indicator','data_col','new_rank_score','higher_is_better','Sub-Pillar']]
agg_df

,Year,Indicator,data_col,new_rank_score,higher_is_better,Sub-Pillar
Country Name,,,,,,
Afghanistan,2019,Internet Exchange Points (IXPs) map,1,1.000000,True,Connectivity Technology
Albania,2019,Internet Exchange Points (IXPs) map,2,1.023148,True,Connectivity Technology
Algeria,2019,Internet Exchange Points (IXPs) map,1,1.000000,True,Connectivity Technology
Angola,2019,Internet Exchange Points (IXPs) map,3,1.046296,True,Connectivity Technology
Argentina,2019,Internet Exchange Points (IXPs) map,33,1.740741,True,Connectivity Technology
...,...,...,...,...,...,...
Uzbekistan,2019,Internet Exchange Points (IXPs) map,3,1.046296,True,Connectivity Technology
Vanuatu,2019,Internet Exchange Points (IXPs) map,1,1.000000,True,Connectivity Technology
Viet Nam,2019,Internet Exchange Points (IXPs) map,3,1.046296,True,Connectivity Technology


In [137]:
agg_df.to_csv('../indicator_scores/infrastructure_{}_scores.csv'.format(indicator), index=False)

In [138]:
### * 18. ISP Speeds - Mobile

In [139]:
indicators[17]

# load data
indicator = indicators[17]
print(indicator)
bf = bnames[bnames['Indicator']==indicator]['Filename'].values[0]
print(bf)

df = pd.read_csv('../../processed/{}.csv'.format(bf))

ISP Speeds - Mobile
mobile_speed


In [140]:
df.head(15)

,#,Year,Country,Mbps
0,1,2021,United Arab Emirates,190.03
1,2,2021,South Korea,189.20
2,3,2021,Qatar,170.77
3,4,2021,China,157.72
4,5,2021,Cyprus,154.13
5,6,2021,Norway,152.53
6,7,2021,Saudi Arabia,151.13
7,8,2021,Kuwait,140.16
8,9,2021,Australia,122.27
9,10,2021,Bulgaria,120.44


In [141]:
subpillars[0]
subpillar = subpillars[0]
print(subpillar)

Connectivity Technology


In [142]:
# create standard columns
# df.rename(columns={'COUNTRY/ECONOMY':'Country Name'}, inplace=True)

df['higher_is_better'] = True
df['Indicator'] = indicator
df['data_col'] = df['Mbps'] 
df['Country Name'] = df['Country']
df['Sub-Pillar'] = subpillar

min_rank = df['data_col'].min()
max_rank = df['data_col'].max()

# transform 0-1 rank into 1-6
df['new_rank_score'] = df['data_col'].apply(lambda row: convert_rank(row,old_min=min_rank,old_max=max_rank))

In [143]:
df = df[['Country Name', 'Year','Indicator','data_col','new_rank_score','higher_is_better','Sub-Pillar']]
df

,Country Name,Year,Indicator,data_col,new_rank_score,higher_is_better,Sub-Pillar
0,United Arab Emirates,2021,ISP Speeds - Mobile,190.03,6.000000,True,Connectivity Technology
1,South Korea,2021,ISP Speeds - Mobile,189.20,5.977362,True,Connectivity Technology
2,Qatar,2021,ISP Speeds - Mobile,170.77,5.474689,True,Connectivity Technology
3,China,2021,ISP Speeds - Mobile,157.72,5.118754,True,Connectivity Technology
4,Cyprus,2021,ISP Speeds - Mobile,154.13,5.020838,True,Connectivity Technology
...,...,...,...,...,...,...,...
134,Bangladesh,2021,ISP Speeds - Mobile,12.60,1.160648,True,Connectivity Technology
135,Zimbabwe,2021,ISP Speeds - Mobile,11.71,1.136374,True,Connectivity Technology
136,Palestine,2021,ISP Speeds - Mobile,7.62,1.024820,True,Connectivity Technology
137,Venezuela,2021,ISP Speeds - Mobile,7.61,1.024547,True,Connectivity Technology


In [144]:
### 19. Individuals using the Internet (% of population)

In [145]:
indicators[18]

# load data
indicator = indicators[18]
print(indicator)
bf = bnames[bnames['Indicator']==indicator]['Filename'].values[0]
print(bf)

df = pd.read_csv('../../processed/{}.csv'.format(bf))

Individuals using the Internet (% of population)
ITU_database


In [146]:
df.head(15)

,Country,Region,ISO,Indicator name,Year,Value,data_country,data_year
0,Angola,Africa,AGO,Female mobile phone ownership as a % of total ...,2010.0,NaN,NaN,NaN
1,Benin,Africa,BEN,Female mobile phone ownership as a % of total ...,2010.0,NaN,NaN,NaN
2,Botswana,Africa,BWA,Female mobile phone ownership as a % of total ...,2010.0,NaN,NaN,NaN
3,Burkina Faso,Africa,BFA,Female mobile phone ownership as a % of total ...,2010.0,NaN,NaN,NaN
4,Burundi,Africa,BDI,Female mobile phone ownership as a % of total ...,2010.0,NaN,NaN,NaN
5,Cabo Verde,Africa,CPV,Female mobile phone ownership as a % of total ...,2010.0,NaN,NaN,NaN
6,Cameroon,Africa,CMR,Female mobile phone ownership as a % of total ...,2010.0,NaN,NaN,NaN
7,Central African Rep.,Africa,CAF,Female mobile phone ownership as a % of total ...,2010.0,NaN,NaN,NaN
8,Chad,Africa,TCD,Female mobile phone ownership as a % of total ...,2010.0,NaN,NaN,NaN
9,Congo (Rep. of the),Africa,COG,Female mobile phone ownership as a % of total ...,2010.0,NaN,NaN,NaN


In [147]:
subpillars[0]
subpillar = subpillars[0]
print(subpillar)

Connectivity Technology


In [148]:
df = df[(df['Indicator name'] == 'Individuals using the Internet, total (%)')]
df = df[(df.Year==2019)]
df.head(15)

,Country,Region,ISO,Indicator name,Year,Value,data_country,data_year
47607,Cabo Verde,Africa,CPV,"Individuals using the Internet, total (%)",2019.0,61.943398,NaN,NaN
47608,Central African Rep.,Africa,CAF,"Individuals using the Internet, total (%)",2019.0,NaN,NaN,NaN
47609,Congo (Rep. of the),Africa,COG,"Individuals using the Internet, total (%)",2019.0,NaN,NaN,NaN
47610,Côte d'Ivoire,Africa,CIV,"Individuals using the Internet, total (%)",2019.0,36.288955,NaN,NaN
47611,Equatorial Guinea,Africa,GNQ,"Individuals using the Internet, total (%)",2019.0,NaN,NaN,NaN
47612,Eritrea,Africa,ERI,"Individuals using the Internet, total (%)",2019.0,NaN,NaN,NaN
47613,Eswatini,Africa,SWZ,"Individuals using the Internet, total (%)",2019.0,NaN,NaN,NaN
47614,Kenya,Africa,KEN,"Individuals using the Internet, total (%)",2019.0,22.565119,NaN,NaN
47615,Lesotho,Africa,LSO,"Individuals using the Internet, total (%)",2019.0,42.301734,NaN,NaN
47616,Madagascar,Africa,MDG,"Individuals using the Internet, total (%)",2019.0,NaN,NaN,NaN


In [149]:
# create standard columns
# df.rename(columns={'COUNTRY/ECONOMY':'Country Name'}, inplace=True)

df['higher_is_better'] = True
df['Indicator'] = indicator
df['data_col'] = df['Value'] 
df.rename(columns = {'Country':'Country Name'}, inplace=True)
df['Sub-Pillar'] = subpillar

min_rank = df['data_col'].min()
max_rank = df['data_col'].max()

# transform 0-1 rank into 1-6
df['new_rank_score'] = df['data_col'].apply(lambda row: convert_rank(row,old_min=0,old_max=100))

In [150]:
df = df[['Country Name', 'Year','Indicator','data_col','new_rank_score','higher_is_better','Sub-Pillar']]
df

,Country Name,Year,Indicator,data_col,new_rank_score,higher_is_better,Sub-Pillar
47607,Cabo Verde,2019.0,Individuals using the Internet (% of population),61.943398,4.097170,True,Connectivity Technology
47608,Central African Rep.,2019.0,Individuals using the Internet (% of population),NaN,NaN,True,Connectivity Technology
47609,Congo (Rep. of the),2019.0,Individuals using the Internet (% of population),NaN,NaN,True,Connectivity Technology
47610,Côte d'Ivoire,2019.0,Individuals using the Internet (% of population),36.288955,2.814448,True,Connectivity Technology
47611,Equatorial Guinea,2019.0,Individuals using the Internet (% of population),NaN,NaN,True,Connectivity Technology
...,...,...,...,...,...,...,...
47739,Saint Vincent and the Grenadines,2019.0,Individuals using the Internet (% of population),NaN,NaN,True,Connectivity Technology
47740,Trinidad and Tobago,2019.0,Individuals using the Internet (% of population),NaN,NaN,True,Connectivity Technology
47741,United States,2019.0,Individuals using the Internet (% of population),89.430285,5.471514,True,Connectivity Technology
47742,Uruguay,2019.0,Individuals using the Internet (% of population),83.351534,5.167577,True,Connectivity Technology


In [151]:
df.to_csv('../indicator_scores/infrastructure_{}_scores.csv'.format(indicator), index=False)

In [152]:
### 20. Mobile-cellular subscriptions per 100 inhabitants

In [153]:
indicators[19]

# load data
indicator = indicators[19]
print(indicator)
bf = bnames[bnames['Indicator']==indicator]['Filename'].values[0]
print(bf)

df = pd.read_csv('../../processed/{}.csv'.format(bf))

# Waiting for internet_speed to be added to the processed folder

Mobile cellular subscriptions (per 100 people)
ITU_database


In [154]:
df.head(15)

,Country,Region,ISO,Indicator name,Year,Value,data_country,data_year
0,Angola,Africa,AGO,Female mobile phone ownership as a % of total ...,2010.0,NaN,NaN,NaN
1,Benin,Africa,BEN,Female mobile phone ownership as a % of total ...,2010.0,NaN,NaN,NaN
2,Botswana,Africa,BWA,Female mobile phone ownership as a % of total ...,2010.0,NaN,NaN,NaN
3,Burkina Faso,Africa,BFA,Female mobile phone ownership as a % of total ...,2010.0,NaN,NaN,NaN
4,Burundi,Africa,BDI,Female mobile phone ownership as a % of total ...,2010.0,NaN,NaN,NaN
5,Cabo Verde,Africa,CPV,Female mobile phone ownership as a % of total ...,2010.0,NaN,NaN,NaN
6,Cameroon,Africa,CMR,Female mobile phone ownership as a % of total ...,2010.0,NaN,NaN,NaN
7,Central African Rep.,Africa,CAF,Female mobile phone ownership as a % of total ...,2010.0,NaN,NaN,NaN
8,Chad,Africa,TCD,Female mobile phone ownership as a % of total ...,2010.0,NaN,NaN,NaN
9,Congo (Rep. of the),Africa,COG,Female mobile phone ownership as a % of total ...,2010.0,NaN,NaN,NaN


In [155]:
subpillars[0]
subpillar = subpillars[0]
print(subpillar)

Connectivity Technology


In [156]:
df = df[(df['Indicator name'] == 'Mobile-cellular subscriptions per 100 inhabitants')]
df = df[(df.Year==2020)]
df.head(15)

,Country,Region,ISO,Indicator name,Year,Value,data_country,data_year
32475,Angola,Africa,AGO,Mobile-cellular subscriptions per 100 inhabitants,2020.0,44.559511,NaN,NaN
32476,Benin,Africa,BEN,Mobile-cellular subscriptions per 100 inhabitants,2020.0,91.897280,NaN,NaN
32477,Botswana,Africa,BWA,Mobile-cellular subscriptions per 100 inhabitants,2020.0,162.399011,NaN,NaN
32478,Burkina Faso,Africa,BFA,Mobile-cellular subscriptions per 100 inhabitants,2020.0,105.807440,NaN,NaN
32479,Burundi,Africa,BDI,Mobile-cellular subscriptions per 100 inhabitants,2020.0,55.767172,NaN,NaN
32480,Cabo Verde,Africa,CPV,Mobile-cellular subscriptions per 100 inhabitants,2020.0,97.975133,NaN,NaN
32481,Cameroon,Africa,CMR,Mobile-cellular subscriptions per 100 inhabitants,2020.0,95.100069,NaN,NaN
32482,Central African Rep.,Africa,CAF,Mobile-cellular subscriptions per 100 inhabitants,2020.0,NaN,NaN,NaN
32483,Chad,Africa,TCD,Mobile-cellular subscriptions per 100 inhabitants,2020.0,52.887026,NaN,NaN
32484,Congo (Rep. of the),Africa,COG,Mobile-cellular subscriptions per 100 inhabitants,2020.0,NaN,NaN,NaN


In [157]:
# create standard columns
# df.rename(columns={'COUNTRY/ECONOMY':'Country Name'}, inplace=True)

df['higher_is_better'] = True
df['Indicator'] = indicator
df['data_col'] = df['Value'] 
df.rename(columns = {'Country':'Country Name'}, inplace=True)
df['Sub-Pillar'] = subpillar

min_rank = df['data_col'].min()
max_rank = df['data_col'].max()

# transform 0-1 rank into 1-6
df['new_rank_score'] = df['data_col'].apply(lambda row: convert_rank(row,old_min=0,old_max=max_rank))

In [158]:
df = df[['Country Name', 'Year','Indicator','data_col','new_rank_score','higher_is_better','Sub-Pillar']]
df

,Country Name,Year,Indicator,data_col,new_rank_score,higher_is_better,Sub-Pillar
32475,Angola,2020.0,Mobile cellular subscriptions (per 100 people),44.559511,1.763912,True,Connectivity Technology
32476,Benin,2020.0,Mobile cellular subscriptions (per 100 people),91.897280,2.575453,True,Connectivity Technology
32477,Botswana,2020.0,Mobile cellular subscriptions (per 100 people),162.399011,3.784108,True,Connectivity Technology
32478,Burkina Faso,2020.0,Mobile cellular subscriptions (per 100 people),105.807440,2.813923,True,Connectivity Technology
32479,Burundi,2020.0,Mobile cellular subscriptions (per 100 people),55.767172,1.956051,True,Connectivity Technology
...,...,...,...,...,...,...,...
32666,Suriname,2020.0,Mobile cellular subscriptions (per 100 people),153.305479,3.628212,True,Connectivity Technology
32667,Trinidad and Tobago,2020.0,Mobile cellular subscriptions (per 100 people),142.051665,3.435280,True,Connectivity Technology
32668,United States,2020.0,Mobile cellular subscriptions (per 100 people),NaN,NaN,True,Connectivity Technology
32669,Uruguay,2020.0,Mobile cellular subscriptions (per 100 people),NaN,NaN,True,Connectivity Technology


In [159]:
df.to_csv('../indicator_scores/infrastructure_{}_scores.csv'.format(indicator), index=False)

In [160]:
### 21. * Average fixed broadband download speeds

In [161]:
indicators[20]

# load data
indicator = indicators[20]
print(indicator)
bf = bnames[bnames['Indicator']==indicator]['Filename'].values[0]
print(bf)

df = pd.read_csv('../../processed/{}.csv'.format(bf))

Average fixed broadband download speeds 
fixed_bdbd_spd_dl_ul


In [162]:
df.head(15)

,country,broadband,mobile,pop2021
0,Singapore,226.60,60.52,5896.686
1,Hong Kong,210.73,50.96,7552.810
2,Romania,193.47,40.35,19127.774
3,Switzerland,178.81,64.10,8715.494
4,Thailand,175.22,34.38,69950.850
5,France,173.05,49.82,65426.179
6,Denmark,162.08,63.86,5813.298
7,Monaco,162.06,NaN,39.511
8,Hungary,161.51,44.52,9634.164
9,United States,161.14,47.13,332915.073


In [163]:
subpillars[0]
subpillar = subpillars[0]
print(subpillar)

Connectivity Technology


In [164]:
# create standard columns

df.rename(columns={'Country':'Country Name'}, inplace=True)
df['higher_is_better'] = True
df['Year'] = 2021
df['Indicator'] = indicator
df['data_col'] = df['broadband'] 
df.rename(columns = {'country':'Country Name'}, inplace=True)
df['Sub-Pillar'] = subpillar

min_rank = df['data_col'].min()
max_rank = df['data_col'].max()

# transform 0-1 rank into 1-6
df['new_rank_score'] = df['data_col'].apply(lambda row: convert_rank(row,old_min=0,old_max=100))

In [165]:
df = df[['Country Name', 'Year','Indicator','data_col','new_rank_score','higher_is_better','Sub-Pillar']]
df

,Country Name,Year,Indicator,data_col,new_rank_score,higher_is_better,Sub-Pillar
0,Singapore,2021,Average fixed broadband download speeds,226.60,12.3300,True,Connectivity Technology
1,Hong Kong,2021,Average fixed broadband download speeds,210.73,11.5365,True,Connectivity Technology
2,Romania,2021,Average fixed broadband download speeds,193.47,10.6735,True,Connectivity Technology
3,Switzerland,2021,Average fixed broadband download speeds,178.81,9.9405,True,Connectivity Technology
4,Thailand,2021,Average fixed broadband download speeds,175.22,9.7610,True,Connectivity Technology
...,...,...,...,...,...,...,...
167,Mauritania,2021,Average fixed broadband download speeds,6.47,1.3235,True,Connectivity Technology
168,Algeria,2021,Average fixed broadband download speeds,4.81,1.2405,True,Connectivity Technology
169,Yemen,2021,Average fixed broadband download speeds,4.25,1.2125,True,Connectivity Technology
170,Turkmenistan,2021,Average fixed broadband download speeds,3.40,1.1700,True,Connectivity Technology


In [166]:
### 22. ISP internet download speed average

In [167]:
indicators[21]

# load data
indicator = indicators[21]
print(indicator)
bf = bnames[bnames['Indicator']==indicator]['Filename'].values[0]
print(bf)

df = pd.read_csv('../../processed/{}.csv'.format(bf))

ISP internet download speed average 
internet_speed


In [168]:
df.head(15)

,Ranking,Year,Country,Download speed (Mbps)
0,1,2021,Monaco,256.70
1,2,2021,Singapore,256.03
2,3,2021,Hong Kong (SAR),248.59
3,4,2021,Thailand,216.16
4,5,2021,Romania,215.30
5,6,2021,Switzerland,214.82
6,7,2021,South Korea,212.83
7,8,2021,Chile,209.45
8,9,2021,Denmark,208.50
9,10,2021,Liechtenstein,207.44


In [169]:
subpillars[0]
subpillar = subpillars[0]
print(subpillar)

Connectivity Technology


In [170]:
# create standard columns
# df.rename(columns={'COUNTRY/ECONOMY':'Country Name'}, inplace=True)

df['higher_is_better'] = True
df['Indicator'] = indicator
df['data_col'] = df['Download speed (Mbps)'] 
df['Country Name'] = df['Country']
df['Sub-Pillar'] = subpillar

min_rank = df['data_col'].min()
max_rank = df['data_col'].max()

# transform 0-1 rank into 1-6
df['new_rank_score'] = df['data_col'].apply(lambda row: convert_rank(row,old_min=min_rank,old_max=max_rank))

In [171]:
df = df[['Country Name', 'Year','Indicator','data_col','new_rank_score','higher_is_better','Sub-Pillar']]
df

,Country Name,Year,Indicator,data_col,new_rank_score,higher_is_better,Sub-Pillar
0,Monaco,2021,ISP internet download speed average,256.70,6.000000,True,Connectivity Technology
1,Singapore,2021,ISP internet download speed average,256.03,5.986711,True,Connectivity Technology
2,Hong Kong (SAR),2021,ISP internet download speed average,248.59,5.839145,True,Connectivity Technology
3,Thailand,2021,ISP internet download speed average,216.16,5.195922,True,Connectivity Technology
4,Romania,2021,ISP internet download speed average,215.30,5.178865,True,Connectivity Technology
...,...,...,...,...,...,...,...
175,Niger,2021,ISP internet download speed average,8.33,1.073783,True,Connectivity Technology
176,Sudan,2021,ISP internet download speed average,7.48,1.056924,True,Connectivity Technology
177,Yemen,2021,ISP internet download speed average,5.49,1.017454,True,Connectivity Technology
178,Cuba,2021,ISP internet download speed average,4.61,1.000000,True,Connectivity Technology


In [172]:
df.to_csv('../indicator_scores/infrastructure_{}_scores.csv'.format(indicator), index=False)

In [173]:
### 23. Mobile dowload speed at the slowest hour

In [174]:
indicators[22]

# load data
indicator = indicators[22]
print(indicator)
bf = bnames[bnames['Indicator']==indicator]['Filename'].values[0]
print(bf)

df = pd.read_csv('../../processed/{}.csv'.format(bf))

Mobile dowload speed at the slowest hour of the day
mobile_latency


In [175]:
subpillars[0]
subpillar = subpillars[0]
print(subpillar)

Connectivity Technology


In [176]:
# filter most recent year
df = df[(df.Year==2019)]

# create standard columns
# df.rename(columns={'COUNTRY/ECONOMY':'Country Name'}, inplace=True)

df['higher_is_better'] = True
df['Indicator'] = indicator
df['data_col'] = df['Mobile latencies'] 
df['Country Name'] = df['Country']
df['Sub-Pillar'] = subpillar

min_rank = df['data_col'].min()
max_rank = df['data_col'].max()

# transform 0-1 rank into 1-6
df['new_rank_score'] = df['data_col'].apply(lambda row: convert_rank(row,old_min=0,old_max=100))


# invert since to lower rank is better
df['new_rank_score'] = df['new_rank_score'].apply(lambda row: (6-row)+1)

In [177]:
df = df[['Country Name', 'Year','Indicator','data_col','new_rank_score','higher_is_better','Sub-Pillar']]
df

,Country Name,Year,Indicator,data_col,new_rank_score,higher_is_better,Sub-Pillar
5,Afghanistan,2019,Mobile dowload speed at the slowest hour of th...,42.627621,3.868619,True,Connectivity Technology
11,Angola,2019,Mobile dowload speed at the slowest hour of th...,66.285637,2.685718,True,Connectivity Technology
17,Albania,2019,Mobile dowload speed at the slowest hour of th...,84.636459,1.768177,True,Connectivity Technology
23,United Arab Emirates,2019,Mobile dowload speed at the slowest hour of th...,84.951675,1.752416,True,Connectivity Technology
29,Argentina,2019,Mobile dowload speed at the slowest hour of th...,80.727203,1.963640,True,Connectivity Technology
...,...,...,...,...,...,...,...
995,Samoa,2019,Mobile dowload speed at the slowest hour of th...,70.360039,2.481998,True,Connectivity Technology
1001,Yemen,2019,Mobile dowload speed at the slowest hour of th...,9.473684,5.526316,True,Connectivity Technology
1007,South Africa,2019,Mobile dowload speed at the slowest hour of th...,78.438538,2.078073,True,Connectivity Technology
1013,Zambia,2019,Mobile dowload speed at the slowest hour of th...,60.044987,2.997751,True,Connectivity Technology


In [178]:
df.to_csv('../indicator_scores/infrastructure_{}_scores.csv'.format(indicator), index=False)

In [179]:
### * 24. Mobile download speed average

In [180]:
indicators[23]

# load data
indicator = indicators[23]
print(indicator)
bf = bnames[bnames['Indicator']==indicator]['Filename'].values[0]
print(bf)

df = pd.read_csv('../../processed/{}.csv'.format(bf))


Mobile download speed average 
mobile_speed


In [181]:
df.head(15)

,#,Year,Country,Mbps
0,1,2021,United Arab Emirates,190.03
1,2,2021,South Korea,189.20
2,3,2021,Qatar,170.77
3,4,2021,China,157.72
4,5,2021,Cyprus,154.13
5,6,2021,Norway,152.53
6,7,2021,Saudi Arabia,151.13
7,8,2021,Kuwait,140.16
8,9,2021,Australia,122.27
9,10,2021,Bulgaria,120.44


In [182]:
subpillars[0]
subpillar = subpillars[0]
print(subpillar)

Connectivity Technology


In [183]:
# create standard columns
# df.rename(columns={'COUNTRY/ECONOMY':'Country Name'}, inplace=True)

df['higher_is_better'] = True
df['Indicator'] = indicator
df['data_col'] = df['Mbps'] 
df['Country Name'] = df['Country']
df['Sub-Pillar'] = subpillar

min_rank = df['data_col'].min()
max_rank = df['data_col'].max()

# transform 0-1 rank into 1-6
df['new_rank_score'] = df['data_col'].apply(lambda row: convert_rank(row,old_min=0,old_max=100))

In [184]:
df = df[['Country Name', 'Year','Indicator','data_col','new_rank_score','higher_is_better','Sub-Pillar']]
df

,Country Name,Year,Indicator,data_col,new_rank_score,higher_is_better,Sub-Pillar
0,United Arab Emirates,2021,Mobile download speed average,190.03,10.5015,True,Connectivity Technology
1,South Korea,2021,Mobile download speed average,189.20,10.4600,True,Connectivity Technology
2,Qatar,2021,Mobile download speed average,170.77,9.5385,True,Connectivity Technology
3,China,2021,Mobile download speed average,157.72,8.8860,True,Connectivity Technology
4,Cyprus,2021,Mobile download speed average,154.13,8.7065,True,Connectivity Technology
...,...,...,...,...,...,...,...
134,Bangladesh,2021,Mobile download speed average,12.60,1.6300,True,Connectivity Technology
135,Zimbabwe,2021,Mobile download speed average,11.71,1.5855,True,Connectivity Technology
136,Palestine,2021,Mobile download speed average,7.62,1.3810,True,Connectivity Technology
137,Venezuela,2021,Mobile download speed average,7.61,1.3805,True,Connectivity Technology


In [185]:
### 25. Postal Coverage

In [186]:
indicators[24]

# load data
indicator = indicators[24]
print(indicator)
bf = bnames[bnames['Indicator']==indicator]['Filename'].values[0]
print(bf)

df = pd.read_csv('../../processed/{}.csv'.format(bf))

Postal Coverage
postal_coverage


In [187]:
df['2019'] = df['2019'].replace('ND',np.nan)
df['2019'].describe

<bound method NDFrame.describe of 0        50
1      96.5
2        90
3       NaN
4       NaN
       ... 
179     NaN
180      98
181      98
182     NaN
183      32
Name: 2019, Length: 184, dtype: object>

In [188]:
df.head(15)

,2015,2016,2017,2018,2019,Administrations,Items
0,NaN,60,60,50,50,Afghanistan,8.5 Percentage of the population having mail d...
1,NaN,99.7,96.2,NaN,96.5,Albania,8.5 Percentage of the population having mail d...
2,NaN,NaN,100,90,90,Algeria,8.5 Percentage of the population having mail d...
3,"15,00",15,15,21,NaN,Angola,8.5 Percentage of the population having mail d...
4,"80,00",93,40,NaN,NaN,Antigua and Barbuda,8.5 Percentage of the population having mail d...
5,"93,00",NaN,NaN,NaN,NaN,Argentina,8.5 Percentage of the population having mail d...
6,"93,00",100,100,100,100,Armenia,8.5 Percentage of the population having mail d...
7,ND,ND,NaN,ND,NaN,Australia,8.5 Percentage of the population having mail d...
8,"99,00",99,99,99,99,Austria,8.5 Percentage of the population having mail d...
9,ND,ND,NaN,ND,NaN,Azerbaijan,8.5 Percentage of the population having mail d...


In [189]:
subpillars[0]
subpillar = subpillars[0]
print(subpillar)

Connectivity Technology


In [190]:
# create standard columns
# df.rename(columns={'COUNTRY/ECONOMY':'Country Name'}, inplace=True)

df['higher_is_better'] = True
df['Year'] = 2019
df['Indicator'] = indicator
df['data_col'] = df['2019'].astype(float)
df['Sub-Pillar'] = subpillar
df['Country Name'] = df['Administrations']

min_rank = df['data_col'].min()
max_rank = df['data_col'].max()

# transform 0-1 rank into 1-6
df['new_rank_score'] = df['data_col'].apply(lambda row: convert_rank(row,old_min=0,old_max=100))

In [191]:
df = df[['Country Name', 'Year','Indicator','data_col','new_rank_score','higher_is_better','Sub-Pillar']]
df

,Country Name,Year,Indicator,data_col,new_rank_score,higher_is_better,Sub-Pillar
0,Afghanistan,2019,Postal Coverage,50.0,3.500,True,Connectivity Technology
1,Albania,2019,Postal Coverage,96.5,5.825,True,Connectivity Technology
2,Algeria,2019,Postal Coverage,90.0,5.500,True,Connectivity Technology
3,Angola,2019,Postal Coverage,NaN,NaN,True,Connectivity Technology
4,Antigua and Barbuda,2019,Postal Coverage,NaN,NaN,True,Connectivity Technology
...,...,...,...,...,...,...,...
179,Vatican,2019,Postal Coverage,NaN,NaN,True,Connectivity Technology
180,Venezuela (Bolivarian Rep.),2019,Postal Coverage,98.0,5.900,True,Connectivity Technology
181,Viet Nam,2019,Postal Coverage,98.0,5.900,True,Connectivity Technology
182,Zambia,2019,Postal Coverage,NaN,NaN,True,Connectivity Technology


In [192]:
df.to_csv('../indicator_scores/infrastructure_{}_scores.csv'.format(indicator), index=False)

In [193]:
### 26. Logistics Performance Index (LPI) 

In [194]:
indicators[25]

# load data
indicator = indicators[25]
print(indicator)
bf = bnames[bnames['Indicator']==indicator]['Filename'].values[0]
print(bf)

df = pd.read_csv('../../processed/{}.csv'.format(bf))

Logistics Performance Index (LPI) - Infrastructure Rating
logistics_performance_index


In [195]:
df.head(15)

,Country,Code,score,data_country,data_year
0,Germany,DEU,4.20,NaN,NaN
1,Sweden,SWE,4.05,NaN,NaN
2,Belgium,BEL,4.04,NaN,NaN
3,Austria,AUT,4.03,NaN,NaN
4,Japan,JPN,4.03,NaN,NaN
5,Netherlands,NLD,4.02,NaN,NaN
6,Singapore,SGP,4.00,NaN,NaN
7,Denmark,DNK,3.99,NaN,NaN
8,United Kingdom,GBR,3.99,NaN,NaN
9,Finland,FIN,3.97,NaN,NaN


In [196]:
subpillars[0]
subpillar = subpillars[0]
print(subpillar)

Connectivity Technology


In [197]:
# create standard columns
# df.rename(columns={'COUNTRY/ECONOMY':'Country Name'}, inplace=True)

df['higher_is_better'] = True
df['Year'] = 2018
df['Indicator'] = indicator
df['data_col'] = df['score']
df.rename(columns={'Country':'Country Name'}, inplace=True)
df['Sub-Pillar'] = subpillar

min_rank = df['data_col'].min()
max_rank = df['data_col'].max()

# transform 0-1 rank into 1-6
df['new_rank_score'] = df['data_col'].apply(lambda row: convert_rank_c(row,old_min=1,old_max=5))

In [198]:
df = df[['Country Name', 'Year','Indicator','data_col','new_rank_score','higher_is_better','Sub-Pillar']]
df

,Country Name,Year,Indicator,data_col,new_rank_score,higher_is_better,Sub-Pillar
0,Germany,2018,Logistics Performance Index (LPI) - Infrastruc...,4.20,5.0000,True,Connectivity Technology
1,Sweden,2018,Logistics Performance Index (LPI) - Infrastruc...,4.05,4.8125,True,Connectivity Technology
2,Belgium,2018,Logistics Performance Index (LPI) - Infrastruc...,4.04,4.8000,True,Connectivity Technology
3,Austria,2018,Logistics Performance Index (LPI) - Infrastruc...,4.03,4.7875,True,Connectivity Technology
4,Japan,2018,Logistics Performance Index (LPI) - Infrastruc...,4.03,4.7875,True,Connectivity Technology
...,...,...,...,...,...,...,...
155,Sierra Leone,2018,Logistics Performance Index (LPI) - Infrastruc...,2.08,2.3500,True,Connectivity Technology
156,Niger,2018,Logistics Performance Index (LPI) - Infrastruc...,2.07,2.3375,True,Connectivity Technology
157,Burundi,2018,Logistics Performance Index (LPI) - Infrastruc...,2.06,2.3250,True,Connectivity Technology
158,Angola,2018,Logistics Performance Index (LPI) - Infrastruc...,2.05,2.3125,True,Connectivity Technology


In [199]:
df.to_csv('../indicator_scores/infrastructure_{}_scores.csv'.format(indicator), index=False)

In [200]:
### 27. National cyber security index

In [201]:
indicators[26]

# load data
indicator = indicators[26]
print(indicator)
bf = bnames[bnames['Indicator']==indicator]['Filename'].values[0]
print(bf)

df = pd.read_csv('../../processed/{}.csv'.format(bf))

National cyber security index
national_cybersecurity_index


In [202]:
df.head(15)

,Rank,Country,National Cyber Security Index,Difference,Unnamed: 4
0,1.0,Greece,96.10,32.35,NaN
1,2.0,Czech Republic,92.21,23.25,NaN
2,3.0,Estonia,90.91,15.05,NaN
3,4.0,Portugal,89.61,21.76,NaN
4,5.0,Lithuania,88.31,20.01,NaN
5,6.0,Spain,88.31,15.70,NaN
6,7.0,Poland,87.01,21.66,NaN
7,8.0,Belgium,85.71,11.32,NaN
8,9.0,Finland,85.71,6.23,NaN
9,10.0,France,84.42,6.63,NaN


In [203]:
subpillars[0]
subpillar = subpillars[0]
print(subpillar)

Connectivity Technology


In [204]:
# create standard columns
# df.rename(columns={'COUNTRY/ECONOMY':'Country Name'}, inplace=True)

df['higher_is_better'] = True
df['Year'] = 2021
df['Indicator'] = indicator
df['data_col'] = df['National Cyber Security Index']
df.rename(columns={'Country':'Country Name'}, inplace=True)
df['Sub-Pillar'] = subpillar

min_rank = df['data_col'].min()
max_rank = df['data_col'].max()

# transform 0-1 rank into 1-6
df['new_rank_score'] = df['data_col'].apply(lambda row: convert_rank(row,old_min=0,old_max=100))

In [205]:
df = df[['Country Name', 'Year','Indicator','data_col','new_rank_score','higher_is_better','Sub-Pillar']]
df

,Country Name,Year,Indicator,data_col,new_rank_score,higher_is_better,Sub-Pillar
0,Greece,2021,National cyber security index,96.10,5.8050,True,Connectivity Technology
1,Czech Republic,2021,National cyber security index,92.21,5.6105,True,Connectivity Technology
2,Estonia,2021,National cyber security index,90.91,5.5455,True,Connectivity Technology
3,Portugal,2021,National cyber security index,89.61,5.4805,True,Connectivity Technology
4,Lithuania,2021,National cyber security index,88.31,5.4155,True,Connectivity Technology
...,...,...,...,...,...,...,...
156,Burundi,2021,National cyber security index,2.60,1.1300,True,Connectivity Technology
157,Solomon Islands,2021,National cyber security index,2.60,1.1300,True,Connectivity Technology
158,Tuvalu,2021,National cyber security index,2.60,1.1300,True,Connectivity Technology
159,South Sudan,2021,National cyber security index,1.30,1.0650,True,Connectivity Technology


In [206]:
df.to_csv('../indicator_scores/infrastructure_{}_scores.csv'.format(indicator), index=False)

In [207]:
### 28. Global Cybersecurity Index (GCI)

In [208]:
indicators[27]

# load data
indicator = indicators[27]
print(indicator)
bf = bnames[bnames['Indicator']==indicator]['Filename'].values[0]
print(bf)

df = pd.read_csv('../../processed/{}.csv'.format(bf))

Global Cybersecurity Index (GCI)
dice_export_global_cybersecurity_index


In [209]:
df.head(15)

,Country,Score (2020),Rank (2020)
0,United States of America**,100.00,1
1,United Kingdom,99.54,2
2,Saudi Arabia,99.54,2
3,Estonia,99.48,3
4,Korea (Rep. of),98.52,4
5,Singapore,98.52,4
6,Spain,98.52,4
7,Russian Federation,98.06,5
8,United Arab Emirates,98.06,5
9,Malaysia,98.06,5


In [210]:
subpillars[0]
subpillar = subpillars[0]
print(subpillar)

Connectivity Technology


In [211]:
# create standard columns
# df.rename(columns={'COUNTRY/ECONOMY':'Country Name'}, inplace=True)

df['higher_is_better'] = True
df['Year'] = 2020
df['Indicator'] = indicator
df['data_col'] = df['Score (2020)'] 
df.rename(columns={'Country':'Country Name'}, inplace=True)
df['Sub-Pillar'] = subpillar

min_rank = df['data_col'].min()
max_rank = df['data_col'].max()

# transform 0-1 rank into 1-6
df['new_rank_score'] = df['data_col'].apply(lambda row: convert_rank(row,old_min=0,old_max=100))

In [212]:
df = df[['Country Name', 'Year','Indicator','data_col','new_rank_score','higher_is_better','Sub-Pillar']]
df

,Country Name,Year,Indicator,data_col,new_rank_score,higher_is_better,Sub-Pillar
0,United States of America**,2020,Global Cybersecurity Index (GCI),100.00,6.0000,True,Connectivity Technology
1,United Kingdom,2020,Global Cybersecurity Index (GCI),99.54,5.9770,True,Connectivity Technology
2,Saudi Arabia,2020,Global Cybersecurity Index (GCI),99.54,5.9770,True,Connectivity Technology
3,Estonia,2020,Global Cybersecurity Index (GCI),99.48,5.9740,True,Connectivity Technology
4,Korea (Rep. of),2020,Global Cybersecurity Index (GCI),98.52,5.9260,True,Connectivity Technology
...,...,...,...,...,...,...,...
188,Equatorial Guinea**,2020,Global Cybersecurity Index (GCI),1.46,1.0730,True,Connectivity Technology
189,Dem. People's Rep. of Korea**,2020,Global Cybersecurity Index (GCI),1.35,1.0675,True,Connectivity Technology
190,Micronesia*,2020,Global Cybersecurity Index (GCI),0.00,1.0000,True,Connectivity Technology
191,Vatican*,2020,Global Cybersecurity Index (GCI),0.00,1.0000,True,Connectivity Technology


In [213]:
df.to_csv('../indicator_scores/infrastructure_{}_scores.csv'.format(indicator), index=False)

In [214]:
### 29. Software Developer Ecosystem size

In [215]:
indicators[28]

# load data
indicator = indicators[28]
print(indicator)
bf = bnames[bnames['Indicator']==indicator]['Filename'].values[0]
print(bf)

df = pd.read_csv('../../processed/{}.csv'.format(bf))

Software Developer Ecosystem size
software_developer_ecosystem_size


In [216]:
df.head(15)

,Country,GitHub Accounts
0,United States,651017
1,China,183805
2,India,168328
3,United Kingdom,109460
4,Germany,94359
5,Brazil,80903
6,Canada,77318
7,France,66367
8,Russia,58767
9,Australia,41790


In [217]:
subpillars[1]
subpillar = subpillars[1]
print(subpillar)

Innovation Ecosystem


In [218]:
df['higher_is_better'] = True
df['Year'] = 2018
df['Indicator'] = indicator
df['data_col'] = df.iloc[:,[1]]
df.rename(columns={'Country':'Country Name'}, inplace=True)
df['Sub-Pillar'] = subpillar
    
min_rank = df['data_col'].min()
max_rank = df['data_col'].max()

# transform 0-1 rank into 1-6
df['new_rank_score'] = df['data_col'].apply(lambda row: convert_rank(row,old_min=min_rank,old_max=max_rank))

In [219]:
df = df[['Country Name', 'Year','Indicator','data_col','new_rank_score','higher_is_better','Sub-Pillar']]
df

,Country Name,Year,Indicator,data_col,new_rank_score,higher_is_better,Sub-Pillar
0,United States,2018,Software Developer Ecosystem size,651017,6.000000,True,Innovation Ecosystem
1,China,2018,Software Developer Ecosystem size,183805,2.348274,True,Innovation Ecosystem
2,India,2018,Software Developer Ecosystem size,168328,2.227306,True,Innovation Ecosystem
3,United Kingdom,2018,Software Developer Ecosystem size,109460,1.767194,True,Innovation Ecosystem
4,Germany,2018,Software Developer Ecosystem size,94359,1.649165,True,Innovation Ecosystem
5,Brazil,2018,Software Developer Ecosystem size,80903,1.543993,True,Innovation Ecosystem
6,Canada,2018,Software Developer Ecosystem size,77318,1.515973,True,Innovation Ecosystem
7,France,2018,Software Developer Ecosystem size,66367,1.430380,True,Innovation Ecosystem
8,Russia,2018,Software Developer Ecosystem size,58767,1.370978,True,Innovation Ecosystem
9,Australia,2018,Software Developer Ecosystem size,41790,1.238286,True,Innovation Ecosystem


In [220]:
df.to_csv('../indicator_scores/infrastructure_{}_scores.csv'.format(indicator), index=False)

In [221]:
### 30. Digital Work Ecosystem size

In [222]:
indicators[29]

# load data
indicator = indicators[29]
print(indicator)
bf = bnames[bnames['Indicator']==indicator]['Filename'].values[0]
print(bf)

df = pd.read_csv('../../processed/{}.csv'.format(bf))

Digital Work Ecosystem size
digital_platform_economy_index


In [223]:
df.head(15)

,Rank,Country,DPE 2020,GDP per capita 2017
0,1,United States,85.0,54225
1,2,United Kingdom,82.7,39753
2,3,Netherlands,82.4,48473
3,4,Canada,78.2,44018
4,5,Sweden,76.8,46949
5,6,Switzerland,76.3,57410
6,7,Norway,74.4,64800
7,8,Denmark,71.1,46683
8,9,Australia,69.3,44649
9,10,Finland,68.9,40586


In [224]:
subpillars[1]
subpillar = subpillars[1]
print(subpillar)

Innovation Ecosystem


In [225]:
# create standard columns
# df.rename(columns={'COUNTRY/ECONOMY':'Country Name'}, inplace=True)

df['higher_is_better'] = True
df['Year'] = 2020
df['Indicator'] = indicator
df['data_col'] = df.iloc[:,[2]].astype(float)
df.rename(columns={'Country':'Country Name'}, inplace=True)
df['Sub-Pillar'] = subpillar
    
min_rank = df['data_col'].min()
max_rank = df['data_col'].max()

# transform 0-1 rank into 1-6
df['new_rank_score'] = df['data_col'].apply(lambda row: convert_rank(row,old_min=0,old_max=100))

In [226]:
df = df[['Country Name', 'Year','Indicator','data_col','new_rank_score','higher_is_better','Sub-Pillar']]
df

,Country Name,Year,Indicator,data_col,new_rank_score,higher_is_better,Sub-Pillar
0,United States,2020,Digital Work Ecosystem size,85.0,5.250,True,Innovation Ecosystem
1,United Kingdom,2020,Digital Work Ecosystem size,82.7,5.135,True,Innovation Ecosystem
2,Netherlands,2020,Digital Work Ecosystem size,82.4,5.120,True,Innovation Ecosystem
3,Canada,2020,Digital Work Ecosystem size,78.2,4.910,True,Innovation Ecosystem
4,Sweden,2020,Digital Work Ecosystem size,76.8,4.840,True,Innovation Ecosystem
...,...,...,...,...,...,...,...
111,Malawi,2020,Digital Work Ecosystem size,9.8,1.490,True,Innovation Ecosystem
112,Benin,2020,Digital Work Ecosystem size,9.6,1.480,True,Innovation Ecosystem
113,Madagascar,2020,Digital Work Ecosystem size,7.3,1.365,True,Innovation Ecosystem
114,Burundi,2020,Digital Work Ecosystem size,6.9,1.345,True,Innovation Ecosystem


In [227]:
df.to_csv('../indicator_scores/infrastructure_{}_scores.csv'.format(indicator), index=False)

In [228]:
### 31. Country, Industry, Skill Migration Data (Skill) 

In [229]:
indicators[30]

# load data
indicator = indicators[30]
print(indicator)
bf = bnames[bnames['Indicator']==indicator]['Filename'].values[0]
print(bf)

df = pd.read_csv('../../processed/{}.csv'.format(bf))

Country, Industry, Skill Migration Data (Skill)
migration_skill


In [230]:
df.head(15)

,country_code,country_name,wb_income,wb_region,skill_group_id,skill_group_category,skill_group_name,net_per_10K_2015,net_per_10K_2016,net_per_10K_2017,net_per_10K_2018,net_per_10K_2019,Unnamed: 12,data_country,data_year
0,af,Afghanistan,Low income,South Asia,2549.0,Tech Skills,Information Management,-791.59,-705.88,-550.04,-680.92,-1208.79,NaN,NaN,NaN
1,af,Afghanistan,Low income,South Asia,2608.0,Business Skills,Operational Efficiency,-1610.25,-933.55,-776.06,-532.22,-790.09,NaN,NaN,NaN
2,af,Afghanistan,Low income,South Asia,3806.0,Specialized Industry Skills,National Security,-1731.45,-769.68,-756.59,-600.44,-767.64,NaN,NaN,NaN
3,af,Afghanistan,Low income,South Asia,50321.0,Tech Skills,Software Testing,-957.50,-828.54,-964.73,-406.50,-739.51,NaN,NaN,NaN
4,af,Afghanistan,Low income,South Asia,1606.0,Specialized Industry Skills,Navy,-1510.71,-841.17,-842.32,-581.71,-718.64,NaN,NaN,NaN
5,af,Afghanistan,Low income,South Asia,3139.0,Disruptive Tech Skills,Materials Science,-1085.03,-1045.71,-783.03,-473.42,-717.73,NaN,NaN,NaN
6,af,Afghanistan,Low income,South Asia,1315.0,Specialized Industry Skills,Criminal Law,-687.80,-294.93,-769.01,-415.34,-698.84,NaN,NaN,NaN
7,af,Afghanistan,Low income,South Asia,1017.0,Soft Skills,Problem Solving,-906.42,-210.44,-480.92,-126.62,-696.16,NaN,NaN,NaN
8,af,Afghanistan,Low income,South Asia,2130.0,Tech Skills,Software Development Life Cycle (SDLC),-1096.96,-566.70,-478.30,-324.99,-692.24,NaN,NaN,NaN
9,af,Afghanistan,Low income,South Asia,2265.0,Specialized Industry Skills,Cybersecurity,-1046.26,-796.25,-796.72,-517.62,-640.96,NaN,NaN,NaN


In [231]:
subpillars[1]
subpillar = subpillars[1]
print(subpillar)

Innovation Ecosystem


In [232]:
df['Country Name'] = df.iloc[:,[1]]

df_sum = df.groupby('Country Name')[['net_per_10K_2019']].sum()
df_sum

,net_per_10K_2019
Country Name,
Afghanistan,-35926.60
Albania,-20747.41
Algeria,-26729.53
Angola,4332.13
Argentina,-16970.64
...,...
Vietnam,-1952.65
West Bank and Gaza,-13115.14
"Yemen, Rep.",-2249.00


In [233]:
# create standard columns
# df.rename(columns={'COUNTRY/ECONOMY':'Country Name'}, inplace=True)

df_sum['higher_is_better'] = True
df_sum['Year'] = 2019
df_sum['Indicator'] = indicator
df_sum['data_col'] = df_sum['net_per_10K_2019']
df_sum['Sub-Pillar'] = subpillar

min_rank = df_sum['data_col'].min()
max_rank = df_sum['data_col'].max()

# transform 0-1 rank into 1-6
df_sum['new_rank_score'] = df_sum['data_col'].apply(lambda row: convert_rank(row,old_min=min_rank,old_max=max_rank))

In [234]:
df_sum = df_sum[['Year','Indicator','data_col','new_rank_score','higher_is_better','Sub-Pillar']]
df_sum

,Year,Indicator,data_col,new_rank_score,higher_is_better,Sub-Pillar
Country Name,,,,,,
Afghanistan,2019,"Country, Industry, Skill Migration Data (Skill)",-35926.60,4.221400,True,Innovation Ecosystem
Albania,2019,"Country, Industry, Skill Migration Data (Skill)",-20747.41,4.449323,True,Innovation Ecosystem
Algeria,2019,"Country, Industry, Skill Migration Data (Skill)",-26729.53,4.359499,True,Innovation Ecosystem
Angola,2019,"Country, Industry, Skill Migration Data (Skill)",4332.13,4.825905,True,Innovation Ecosystem
Argentina,2019,"Country, Industry, Skill Migration Data (Skill)",-16970.64,4.506034,True,Innovation Ecosystem
...,...,...,...,...,...,...
Vietnam,2019,"Country, Industry, Skill Migration Data (Skill)",-1952.65,4.731536,True,Innovation Ecosystem
West Bank and Gaza,2019,"Country, Industry, Skill Migration Data (Skill)",-13115.14,4.563926,True,Innovation Ecosystem
"Yemen, Rep.",2019,"Country, Industry, Skill Migration Data (Skill)",-2249.00,4.727086,True,Innovation Ecosystem


In [235]:
### 32. Country, Industry, Skill Migration Data (Industry)

In [236]:
indicators[31]

# load data
indicator = indicators[31]
print(indicator)
bf = bnames[bnames['Indicator']==indicator]['Filename'].values[0]
print(bf)

df = pd.read_csv('../../processed/{}.csv'.format(bf))


Country, Industry, Skill Migration Data (Industry)
migration_industry


In [237]:
df.head(15)

,country_code,country_name,wb_income,wb_region,isic_section_index,isic_section_name,industry_id,industry_name,net_per_10K_2015,net_per_10K_2016,net_per_10K_2017,net_per_10K_2018,net_per_10K_2019,Unnamed: 13,data_country,data_year
0,ae,United Arab Emirates,High income,Middle East & North Africa,C,Manufacturing,1,Defense & Space,378.74,127.94,8.20,68.51,49.55,NaN,NaN,NaN
1,ae,United Arab Emirates,High income,Middle East & North Africa,J,Information and communication,3,Computer Hardware,100.97,358.14,112.98,149.57,182.22,NaN,NaN,NaN
2,ae,United Arab Emirates,High income,Middle East & North Africa,J,Information and communication,4,Computer Software,1079.36,848.15,596.48,409.18,407.41,NaN,NaN,NaN
3,ae,United Arab Emirates,High income,Middle East & North Africa,J,Information and communication,5,Computer Networking,401.46,447.39,163.99,236.69,188.07,NaN,NaN,NaN
4,ae,United Arab Emirates,High income,Middle East & North Africa,J,Information and communication,6,Internet,1840.33,1368.42,877.71,852.39,519.40,NaN,NaN,NaN
5,ae,United Arab Emirates,High income,Middle East & North Africa,J,Information and communication,8,Telecommunications,676.91,676.11,365.96,283.59,281.91,NaN,NaN,NaN
6,ae,United Arab Emirates,High income,Middle East & North Africa,M,Professional scientific and technical activities,9,Law Practice,694.50,529.50,362.83,356.56,147.83,NaN,NaN,NaN
7,ae,United Arab Emirates,High income,Middle East & North Africa,M,Professional scientific and technical activities,10,Legal Services,661.10,665.65,462.30,475.62,255.21,NaN,NaN,NaN
8,ae,United Arab Emirates,High income,Middle East & North Africa,M,Professional scientific and technical activities,11,Management Consulting,982.85,694.25,520.28,574.60,406.03,NaN,NaN,NaN
9,ae,United Arab Emirates,High income,Middle East & North Africa,M,Professional scientific and technical activities,12,Biotechnology,1230.17,846.28,500.77,205.78,513.23,NaN,NaN,NaN


In [238]:
subpillars[1]
subpillar = subpillars[1]
print(subpillar)

Innovation Ecosystem


In [239]:
df['Country Name'] = df.iloc[:,[1]]

df_sum = df.groupby('Country Name')[['net_per_10K_2019']].sum()
df_sum

,net_per_10K_2019
Country Name,
Afghanistan,-1369.00
Albania,-2188.08
Algeria,-3243.78
Angola,544.66
Argentina,-4122.87
...,...
Vietnam,-2645.19
West Bank and Gaza,-748.05
"Yemen, Rep.",160.41


In [240]:
## create standard columns
# df.rename(columns={'COUNTRY/ECONOMY':'Country Name'}, inplace=True)

df_sum['higher_is_better'] = True
df_sum['Year'] = 2019
df_sum['Indicator'] = indicator
df_sum['data_col'] = df_sum['net_per_10K_2019']
df_sum['Sub-Pillar'] = subpillar

min_rank = df_sum['data_col'].min()
max_rank = df_sum['data_col'].max()

# transform 0-1 rank into 1-6
df_sum['new_rank_score'] = df_sum['data_col'].apply(lambda row: convert_rank(row,old_min=min_rank,old_max=max_rank))

In [241]:
df_sum = df_sum[['Year','Indicator','data_col','new_rank_score','higher_is_better','Sub-Pillar']]
df_sum

,Year,Indicator,data_col,new_rank_score,higher_is_better,Sub-Pillar
Country Name,,,,,,
Afghanistan,2019,"Country, Industry, Skill Migration Data (Indus...",-1369.00,4.082767,True,Innovation Ecosystem
Albania,2019,"Country, Industry, Skill Migration Data (Indus...",-2188.08,4.016637,True,Innovation Ecosystem
Algeria,2019,"Country, Industry, Skill Migration Data (Indus...",-3243.78,3.931403,True,Innovation Ecosystem
Angola,2019,"Country, Industry, Skill Migration Data (Indus...",544.66,4.237269,True,Innovation Ecosystem
Argentina,2019,"Country, Industry, Skill Migration Data (Indus...",-4122.87,3.860429,True,Innovation Ecosystem
...,...,...,...,...,...,...
Vietnam,2019,"Country, Industry, Skill Migration Data (Indus...",-2645.19,3.979732,True,Innovation Ecosystem
West Bank and Gaza,2019,"Country, Industry, Skill Migration Data (Indus...",-748.05,4.132900,True,Innovation Ecosystem
"Yemen, Rep.",2019,"Country, Industry, Skill Migration Data (Indus...",160.41,4.206246,True,Innovation Ecosystem


In [242]:
### 33. Country, Industry, Skill Migration Data (Nation)

In [243]:
indicators[32]

# load data
indicator = indicators[32]
print(indicator)
bf = bnames[bnames['Indicator']==indicator]['Filename'].values[0]
print(bf)

df = pd.read_csv('../../processed/{}.csv'.format(bf))

Country, Industry, Skill Migration Data (Country)
migration_country


In [244]:
df.head(15)

,base_country_code,base_country_name,base_lat,base_long,base_country_wb_income,base_country_wb_region,target_country_code,target_country_name,target_lat,target_long,target_country_wb_income,target_country_wb_region,net_per_10K_2015,net_per_10K_2016,net_per_10K_2017,net_per_10K_2018,net_per_10K_2019,Unnamed: 17,data_country,data_year
0,ae,United Arab Emirates,23.424076,53.847818,High Income,Middle East & North Africa,af,Afghanistan,33.939110,67.709953,Low Income,South Asia,0.19,0.16,0.11,-0.05,-0.02,NaN,NaN,NaN
1,ae,United Arab Emirates,23.424076,53.847818,High Income,Middle East & North Africa,dz,Algeria,28.033886,1.659626,Upper Middle Income,Middle East & North Africa,0.19,0.25,0.57,0.55,0.78,NaN,NaN,NaN
2,ae,United Arab Emirates,23.424076,53.847818,High Income,Middle East & North Africa,ao,Angola,-11.202692,17.873887,Lower Middle Income,Sub-Saharan Africa,-0.01,0.04,0.11,-0.02,-0.06,NaN,NaN,NaN
3,ae,United Arab Emirates,23.424076,53.847818,High Income,Middle East & North Africa,ar,Argentina,-38.416097,-63.616672,High Income,Latin America & Caribbean,0.16,0.18,0.04,0.01,0.23,NaN,NaN,NaN
4,ae,United Arab Emirates,23.424076,53.847818,High Income,Middle East & North Africa,am,Armenia,40.069099,45.038189,Upper Middle Income,Europe & Central Asia,0.10,0.05,0.03,-0.01,0.02,NaN,NaN,NaN
5,ae,United Arab Emirates,23.424076,53.847818,High Income,Middle East & North Africa,au,Australia,-25.274398,133.775136,High Income,East Asia & Pacific,-1.06,-3.31,-4.01,-4.58,-4.09,NaN,NaN,NaN
6,ae,United Arab Emirates,23.424076,53.847818,High Income,Middle East & North Africa,at,Austria,47.516231,14.550072,High Income,Europe & Central Asia,0.11,-0.08,-0.07,-0.05,-0.16,NaN,NaN,NaN
7,ae,United Arab Emirates,23.424076,53.847818,High Income,Middle East & North Africa,az,Azerbaijan,40.143105,47.576927,Upper Middle Income,Europe & Central Asia,0.24,0.25,0.10,0.05,0.04,NaN,NaN,NaN
8,ae,United Arab Emirates,23.424076,53.847818,High Income,Middle East & North Africa,bh,Bahrain,25.930414,50.637772,High Income,Middle East & North Africa,0.90,0.89,0.38,-0.07,0.15,NaN,NaN,NaN
9,ae,United Arab Emirates,23.424076,53.847818,High Income,Middle East & North Africa,bd,Bangladesh,23.684994,90.356331,Lower Middle Income,South Asia,-0.21,-0.21,-0.26,-0.18,-0.04,NaN,NaN,NaN


In [245]:
subpillars[1]
subpillar = subpillars[1]
print(subpillar)

Innovation Ecosystem


In [246]:
df['Country Name'] = df.iloc[:,[1]]

df_sum = df.groupby('Country Name')[['net_per_10K_2019']].sum()
df_sum

,net_per_10K_2019
Country Name,
Afghanistan,23.01
Albania,-16.58
Algeria,-23.47
Angola,19.07
Argentina,-8.77
...,...
Vietnam,-7.94
West Bank and Gaza,-15.59
"Yemen, Rep.",2.89


In [247]:
## create standard columns
# df.rename(columns={'COUNTRY/ECONOMY':'Country Name'}, inplace=True)

df_sum['higher_is_better'] = True
df_sum['Year'] = 2019
df_sum['Indicator'] = indicator
df_sum['data_col'] = df_sum['net_per_10K_2019']
df_sum ['Sub-Pillar'] = subpillar

min_rank = df_sum['data_col'].min()
max_rank = df_sum['data_col'].max()

# transform 0-1 rank into 1-6
df_sum['new_rank_score'] = df_sum['data_col'].apply(lambda row: convert_rank(row,old_min=min_rank,old_max=max_rank))

In [248]:
df_sum = df_sum[['Year','Indicator','data_col','new_rank_score','higher_is_better','Sub-Pillar']]
df_sum

,Year,Indicator,data_col,new_rank_score,higher_is_better,Sub-Pillar
Country Name,,,,,,
Afghanistan,2019,"Country, Industry, Skill Migration Data (Country)",23.01,3.194208,True,Innovation Ecosystem
Albania,2019,"Country, Industry, Skill Migration Data (Country)",-16.58,2.634379,True,Innovation Ecosystem
Algeria,2019,"Country, Industry, Skill Migration Data (Country)",-23.47,2.536950,True,Innovation Ecosystem
Angola,2019,"Country, Industry, Skill Migration Data (Country)",19.07,3.138494,True,Innovation Ecosystem
Argentina,2019,"Country, Industry, Skill Migration Data (Country)",-8.77,2.744817,True,Innovation Ecosystem
...,...,...,...,...,...,...
Vietnam,2019,"Country, Industry, Skill Migration Data (Country)",-7.94,2.756554,True,Innovation Ecosystem
West Bank and Gaza,2019,"Country, Industry, Skill Migration Data (Country)",-15.59,2.648378,True,Innovation Ecosystem
"Yemen, Rep.",2019,"Country, Industry, Skill Migration Data (Country)",2.89,2.909698,True,Innovation Ecosystem


In [249]:
### 34. Digital Finance Ecosystem size

In [250]:
indicators[33]

# load data
indicator = indicators[33]
print(indicator)
bf = bnames[bnames['Indicator']==indicator]['Filename'].values[0]
print(bf)

df = pd.read_csv('../../processed/{}.csv'.format(bf))

Digital Finance Ecosystem size
global_fintech_ranking


In [251]:
df.head(15)

,Rank,Country,Total Score (2021)
0,1,United States,31.789
1,2,United Kingdom,23.262
2,3,Singapore,19.176
3,4,Lithuania,17.343
4,5,Switzerland,16.018
5,6,The Netherlands,14.464
6,7,Sweden,14.272
7,8,Australia,13.555
8,9,Canada,13.322
9,10,Estonia,13.303


In [252]:
subpillars[1]
subpillar = subpillars[1]
print(subpillar)

Innovation Ecosystem


In [253]:
# create standard columns
# df.rename(columns={'COUNTRY/ECONOMY':'Country Name'}, inplace=True)

df['higher_is_better'] = True
df['Year'] = 2021
df['Indicator'] = indicator
df['data_col'] = df.iloc[:,[2]]
df['Country Name'] = df.iloc[:,[1]]
df['Sub-Pillar'] = subpillar

min_rank = df['data_col'].min()
max_rank = df['data_col'].max()

# transform 0-1 rank into 1-6
df['new_rank_score'] = df['data_col'].apply(lambda row: convert_rank(row,old_min=0,old_max=100))

# Have to shorten the column names to see if this works

In [254]:
df = df[['Country Name', 'Year','Indicator','data_col','new_rank_score','higher_is_better','Sub-Pillar']]
df

,Country Name,Year,Indicator,data_col,new_rank_score,higher_is_better,Sub-Pillar
0,United States,2021,Digital Finance Ecosystem size,31.789,2.58945,True,Innovation Ecosystem
1,United Kingdom,2021,Digital Finance Ecosystem size,23.262,2.16310,True,Innovation Ecosystem
2,Singapore,2021,Digital Finance Ecosystem size,19.176,1.95880,True,Innovation Ecosystem
3,Lithuania,2021,Digital Finance Ecosystem size,17.343,1.86715,True,Innovation Ecosystem
4,Switzerland,2021,Digital Finance Ecosystem size,16.018,1.80090,True,Innovation Ecosystem
...,...,...,...,...,...,...,...
60,Bangladesh,2021,Digital Finance Ecosystem size,5.073,1.25365,True,Innovation Ecosystem
61,Pakistan,2021,Digital Finance Ecosystem size,4.675,1.23375,True,Innovation Ecosystem
62,Uruguay,2021,Digital Finance Ecosystem size,4.562,1.22810,True,Innovation Ecosystem
63,Uganda,2021,Digital Finance Ecosystem size,4.037,1.20185,True,Innovation Ecosystem


In [255]:
df.to_csv('../indicator_scores/infrastructure_{}_scores.csv'.format(indicator))

In [256]:
### 36. Banking Ecosystem size

In [257]:
indicators[35]

# load data
indicator = indicators[35]
print(indicator)
bf = bnames[bnames['Indicator']==indicator]['Filename'].values[0]
print(bf)

df = pd.read_csv('../../processed/{}.csv'.format(bf))

Banking Ecosystem size
banking_sector_size


In [258]:
df.head(15)

#There are two problems:
#1. The column names need some overhaul (moving the second column to the top and make it the column names)
#2. This is a list of cities, not countries, may present probkems later. 

,Country,"Bank assets, percent of GDP, 2017"
0,Hong Kong,256.63
1,China,174.54
2,Denmark,172.28
3,New Zealand,157.73
4,Japan,157.51
5,Lebanon,157.18
6,Singapore,151.67
7,Qatar,149.95
8,South Korea,141.52
9,Australia,140.47


In [259]:
### 36. Banking Ecosystem size

In [260]:
indicators[35]

# load data
indicator = indicators[35]
print(indicator)
bf = bnames[bnames['Indicator']==indicator]['Filename'].values[0]
print(bf)

df = pd.read_csv('../../processed/{}.csv'.format(bf))

Banking Ecosystem size
banking_sector_size


In [261]:
df.head(15)

,Country,"Bank assets, percent of GDP, 2017"
0,Hong Kong,256.63
1,China,174.54
2,Denmark,172.28
3,New Zealand,157.73
4,Japan,157.51
5,Lebanon,157.18
6,Singapore,151.67
7,Qatar,149.95
8,South Korea,141.52
9,Australia,140.47


In [262]:
subpillars[1]
subpillar = subpillars[1]
print(subpillar)

Innovation Ecosystem


In [263]:
# create standard columns
# df.rename(columns={'COUNTRY/ECONOMY':'Country Name'}, inplace=True)

df['higher_is_better'] = True
df['Year'] = 2017
df['Indicator'] = indicator
df['data_col'] = df['Bank assets, percent of GDP, 2017']
df.rename(columns={'Country':'Country Name'}, inplace=True)
df['Sub-Pillar'] = subpillar

min_rank = df['data_col'].min()
max_rank = df['data_col'].max()

# transform 0-1 rank into 1-6
df['new_rank_score'] = df['data_col'].apply(lambda row: convert_rank(row,old_min=0,old_max=max_rank))

In [264]:
df = df[['Country Name', 'Year','Indicator','data_col','new_rank_score','higher_is_better','Sub-Pillar']]
df


,Country Name,Year,Indicator,data_col,new_rank_score,higher_is_better,Sub-Pillar
0,Hong Kong,2017,Banking Ecosystem size,256.63,6.000000,True,Innovation Ecosystem
1,China,2017,Banking Ecosystem size,174.54,4.400616,True,Innovation Ecosystem
2,Denmark,2017,Banking Ecosystem size,172.28,4.356583,True,Innovation Ecosystem
3,New Zealand,2017,Banking Ecosystem size,157.73,4.073101,True,Innovation Ecosystem
4,Japan,2017,Banking Ecosystem size,157.51,4.068815,True,Innovation Ecosystem
...,...,...,...,...,...,...,...
157,Sierra Leone,2017,Banking Ecosystem size,15.06,1.293419,True,Innovation Ecosystem
158,C.A. Republic,2017,Banking Ecosystem size,14.26,1.277832,True,Innovation Ecosystem
159,Sudan,2017,Banking Ecosystem size,11.86,1.231072,True,Innovation Ecosystem
160,Afghanistan,2017,Banking Ecosystem size,3.43,1.066828,True,Innovation Ecosystem


In [265]:
### 37. Angel Ecosystem size

In [266]:
indicators[36]

# load data
indicator = indicators[36]
print(indicator)
bf = bnames[bnames['Indicator']==indicator]['Filename'].values[0]
print(bf)

df = pd.read_csv('../../processed/{}.csv'.format(bf))

Angel Ecosystem size
angel_investment


In [267]:
df.head(15)

,Country,Business angel investments (in million Euros)
0,United Kingdom,153.08
1,Germany,81.77
2,Spain,78.66
3,Finland,54.00
4,Sweden,44.08
5,France,43.00
6,Denmark,33.34
7,Switzerland,32.60
8,Italy,28.20
9,Russia,26.08


In [268]:
subpillars[1]
subpillar = subpillars[1]
print(subpillar)

Innovation Ecosystem


In [269]:
# create standard columns
# df.rename(columns={'COUNTRY/ECONOMY':'Country Name'}, inplace=True)

df['higher_is_better'] = True
df['Year'] = 2019
df['Indicator'] = indicator
df['data_col'] = df['Business angel investments (in million Euros)']
df.rename(columns={'Country':'Country Name'}, inplace=True)
df['Sub-Pillar'] = subpillar

min_rank = df['data_col'].min()
max_rank = df['data_col'].max()

# transform 0-1 rank into 1-6
df['new_rank_score'] = df['data_col'].apply(lambda row: convert_rank(row,old_min=min_rank,old_max=max_rank))

In [270]:
df = df[['Country Name', 'Year','Indicator','data_col','new_rank_score','higher_is_better','Sub-Pillar']]
df

# Okay but there are only EU countries

,Country Name,Year,Indicator,data_col,new_rank_score,higher_is_better,Sub-Pillar
0,United Kingdom,2019,Angel Ecosystem size,153.08,6.000000,True,Innovation Ecosystem
1,Germany,2019,Angel Ecosystem size,81.77,3.666099,True,Innovation Ecosystem
2,Spain,2019,Angel Ecosystem size,78.66,3.564312,True,Innovation Ecosystem
3,Finland,2019,Angel Ecosystem size,54.00,2.757217,True,Innovation Ecosystem
4,Sweden,2019,Angel Ecosystem size,44.08,2.432546,True,Innovation Ecosystem
5,France,2019,Angel Ecosystem size,43.00,2.397198,True,Innovation Ecosystem
6,Denmark,2019,Angel Ecosystem size,33.34,2.081037,True,Innovation Ecosystem
7,Switzerland,2019,Angel Ecosystem size,32.60,2.056817,True,Innovation Ecosystem
8,Italy,2019,Angel Ecosystem size,28.20,1.912810,True,Innovation Ecosystem
9,Russia,2019,Angel Ecosystem size,26.08,1.843425,True,Innovation Ecosystem


In [271]:
### 38. Startup Ecosystem size

In [272]:
indicators[37]

# load data
indicator = indicators[37]
print(indicator)
bf = bnames[bnames['Indicator']==indicator]['Filename'].values[0]
print(bf)

df = pd.read_csv('../../processed/{}.csv'.format(bf))

Startup Ecosystem size
startup_eco_size


In [273]:
df.head(15)

# This is a list of cities, not countries, wonder if it will work 

,Location,Ranking (2019)
0,Silicon Valley,1
1,New York City,2 (tie)
2,London,2 (tie)
3,Beijing,4
4,Boston,5
5,Tel Aviv - Jerusalem,6 (tie)
6,Los Angeles,6 (tie)
7,Shanghai,8
8,Seattle,9
9,Stockholm,10


In [274]:
### 39. Venture Ecosystem size

indicators[38]

# load data
indicator = indicators[38]
print(indicator)
bf = bnames[bnames['Indicator']==indicator]['Filename'].values[0]
print(bf)

df = pd.read_csv('../../processed/{}.csv'.format(bf))

Venture Ecosystem size
startup_eco_size


In [275]:
df.head(15)

# Similar to the previous one

,Location,Ranking (2019)
0,Silicon Valley,1
1,New York City,2 (tie)
2,London,2 (tie)
3,Beijing,4
4,Boston,5
5,Tel Aviv - Jerusalem,6 (tie)
6,Los Angeles,6 (tie)
7,Shanghai,8
8,Seattle,9
9,Stockholm,10


In [276]:
### 40. International Co-Inventions

In [277]:
indicators[39]

# load data
indicator = indicators[39]
print(indicator)
bf = bnames[bnames['Indicator']==indicator]['Filename'].values[0]
print(bf)

df = pd.read_csv('../../processed/{}.csv'.format(bf))

International Co-Inventions
international_co_inventions


In [278]:
df.head(15)

,KINDPATENT,Patent office,KINDCOOP,Type of International Cooperation in Patenting,LOCATION,Country,PARTNER,Partner Country,KINDDATE,Reference Date,...,Time,Unit Code,Unit,PowerCode Code,PowerCode,Reference Period Code,Reference Period,Value,Flag Codes,Flags
0,EPO_A,Patent applications to the EPO,FOR_APP,Foreign ownership of domestic inventions,AUS,Australia,TOTAL,Total Patents,PRIORITY,Priority date,...,1999,NBR,Number,0,Units,NaN,NaN,1082.0,NaN,NaN
1,EPO_A,Patent applications to the EPO,FOR_APP,Foreign ownership of domestic inventions,AUS,Australia,TOTAL,Total Patents,PRIORITY,Priority date,...,2000,NBR,Number,0,Units,NaN,NaN,1159.0,NaN,NaN
2,EPO_A,Patent applications to the EPO,FOR_APP,Foreign ownership of domestic inventions,AUS,Australia,TOTAL,Total Patents,PRIORITY,Priority date,...,2001,NBR,Number,0,Units,NaN,NaN,1119.0,NaN,NaN
3,EPO_A,Patent applications to the EPO,FOR_APP,Foreign ownership of domestic inventions,AUS,Australia,TOTAL,Total Patents,PRIORITY,Priority date,...,2002,NBR,Number,0,Units,NaN,NaN,1193.0,NaN,NaN
4,EPO_A,Patent applications to the EPO,FOR_APP,Foreign ownership of domestic inventions,AUS,Australia,TOTAL,Total Patents,PRIORITY,Priority date,...,2003,NBR,Number,0,Units,NaN,NaN,1238.0,NaN,NaN
5,EPO_A,Patent applications to the EPO,FOR_APP,Foreign ownership of domestic inventions,AUS,Australia,TOTAL,Total Patents,PRIORITY,Priority date,...,2004,NBR,Number,0,Units,NaN,NaN,1336.0,NaN,NaN
6,EPO_A,Patent applications to the EPO,FOR_APP,Foreign ownership of domestic inventions,AUS,Australia,TOTAL,Total Patents,PRIORITY,Priority date,...,2005,NBR,Number,0,Units,NaN,NaN,1332.0,NaN,NaN
7,EPO_A,Patent applications to the EPO,FOR_APP,Foreign ownership of domestic inventions,AUS,Australia,TOTAL,Total Patents,PRIORITY,Priority date,...,2006,NBR,Number,0,Units,NaN,NaN,1219.0,NaN,NaN
8,EPO_A,Patent applications to the EPO,FOR_APP,Foreign ownership of domestic inventions,AUS,Australia,TOTAL,Total Patents,PRIORITY,Priority date,...,2007,NBR,Number,0,Units,NaN,NaN,1131.0,NaN,NaN
9,EPO_A,Patent applications to the EPO,FOR_APP,Foreign ownership of domestic inventions,AUS,Australia,TOTAL,Total Patents,PRIORITY,Priority date,...,2008,NBR,Number,0,Units,NaN,NaN,1110.0,NaN,NaN


In [279]:
subpillars[1]
subpillar = subpillars[1]
print(subpillar)

Innovation Ecosystem


In [280]:
# filter most recent year and global value
df = df[(df.Time==2017)]
df = df[(df.Country!='World')]
df ['Country Name'] = df ['Country']
df

,KINDPATENT,Patent office,KINDCOOP,Type of International Cooperation in Patenting,LOCATION,Country,PARTNER,Partner Country,KINDDATE,Reference Date,...,Unit Code,Unit,PowerCode Code,PowerCode,Reference Period Code,Reference Period,Value,Flag Codes,Flags,Country Name
36,EPO_A,Patent applications to the EPO,FOR_APP,Foreign ownership of domestic inventions,AUS,Australia,TOTAL,Total Patents,APPLICATION,Application date,...,NBR,Number,0,Units,NaN,NaN,1077.0000,NaN,NaN,Australia
73,EPO_A,Patent applications to the EPO,FOR_APP,Foreign ownership of domestic inventions,AUS,Australia,WRD,Total co-operation with abroad,APPLICATION,Application date,...,NBR,Number,0,Units,NaN,NaN,277.0000,NaN,NaN,Australia
110,EPO_A,Patent applications to the EPO,FOR_APP,Foreign ownership of domestic inventions,AUS,Australia,JPN,Japan,APPLICATION,Application date,...,NBR,Number,0,Units,NaN,NaN,11.0000,NaN,NaN,Australia
147,EPO_A,Patent applications to the EPO,FOR_APP,Foreign ownership of domestic inventions,AUS,Australia,USA,United States,APPLICATION,Application date,...,NBR,Number,0,Units,NaN,NaN,137.0000,NaN,NaN,Australia
184,EPO_A,Patent applications to the EPO,FOR_APP,Foreign ownership of domestic inventions,AUS,Australia,EU27,European Union (27),APPLICATION,Application date,...,NBR,Number,0,Units,NaN,NaN,62.0000,NaN,NaN,Australia
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27024,USPTO_A,Patent applications to the USPTO,CO_INV_S,% of patents with foreign co-inventor(s),USA,United States,EU27,European Union (27),APPLICATION,Application date,...,PC,Percentage,0,Units,NaN,NaN,3.6198,NaN,NaN,United States
27062,USPTO_A,Patent applications to the USPTO,CO_INV_S,% of patents with foreign co-inventor(s),EU27,European Union (27 countries),WRD,Total co-operation with abroad,APPLICATION,Application date,...,PC,Percentage,0,Units,NaN,NaN,20.9912,NaN,NaN,European Union (27 countries)
27081,USPTO_A,Patent applications to the USPTO,CO_INV_S,% of patents with foreign co-inventor(s),EU27,European Union (27 countries),JPN,Japan,APPLICATION,Application date,...,PC,Percentage,0,Units,NaN,NaN,0.7465,NaN,NaN,European Union (27 countries)
27100,USPTO_A,Patent applications to the USPTO,CO_INV_S,% of patents with foreign co-inventor(s),EU27,European Union (27 countries),USA,United States,APPLICATION,Application date,...,PC,Percentage,0,Units,NaN,NaN,13.4261,NaN,NaN,European Union (27 countries)


In [281]:
# Create summarization
df_sum = df.groupby('Country Name')[['Value']].sum()
df_sum

,Value
Country Name,
Australia,3.116507e+04
Canada,8.628640e+04
European Union (27 countries),7.976543e+05
European Union (28 countries),0.000000e+00
Japan,5.457706e+05
Netherlands,7.438852e+04
United States,1.224357e+06


In [282]:
# create standard columns
# df.rename(columns={'COUNTRY/ECONOMY':'Country Name'}, inplace=True)

df_sum['higher_is_better'] = True
df_sum['Year'] = 2017
df_sum['Indicator'] = indicator
df_sum['data_col'] = df_sum.iloc[:,[0]]
df_sum['Sub-Pillar'] = subpillar

min_rank = df_sum['data_col'].min()
max_rank = df_sum['data_col'].max()

# transform 0-1 rank into 1-6
df_sum['new_rank_score'] = df_sum['data_col'].apply(lambda row: convert_rank(row,old_min=min_rank,old_max=max_rank))

In [283]:
df_sum = df_sum[['Year','Indicator','data_col','new_rank_score','higher_is_better','Sub-Pillar']]
df_sum
# Okay but the data is too general with many missing countries

,Year,Indicator,data_col,new_rank_score,higher_is_better,Sub-Pillar
Country Name,,,,,,
Australia,2017,International Co-Inventions,3.116507e+04,1.127271,True,Innovation Ecosystem
Canada,2017,International Co-Inventions,8.628640e+04,1.352374,True,Innovation Ecosystem
European Union (27 countries),2017,International Co-Inventions,7.976543e+05,4.257441,True,Innovation Ecosystem
European Union (28 countries),2017,International Co-Inventions,0.000000e+00,1.000000,True,Innovation Ecosystem
Japan,2017,International Co-Inventions,5.457706e+05,3.228805,True,Innovation Ecosystem
Netherlands,2017,International Co-Inventions,7.438852e+04,1.303786,True,Innovation Ecosystem
United States,2017,International Co-Inventions,1.224357e+06,6.000000,True,Innovation Ecosystem


In [284]:
### 41. Availability of Scientists and Enginners

In [285]:
indicators[40]

# load data
indicator = indicators[40]
print(indicator)
bf = bnames[bnames['Indicator']==indicator]['Filename'].values[0]
print(bf)

df = pd.read_csv('../../processed/{}.csv'.format(bf))

Availability of scientists and engineers
TCdata360


In [286]:
df.head(15)

,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,...,2008-2009,2009-2010,2010-2011,2011-2012,2012-2013,2013-2014,2014-2015,2015-2016,2016-2017,2017-2018
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,40.900000,54.127067,NaN,40.920500,38.903010,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,66.000000,49.000000,NaN,87.000000,92.000000,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,3.858249,3.165548,NaN,2.823529,2.601164,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,127.000000,142.000000,NaN,148.000000,144.000000,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,2.797549,2.242699,NaN,2.028571,2.177586,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,130.000000,142.000000,NaN,148.000000,144.000000,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,2.196592,2.090002,NaN,2.030303,2.249658,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,138.000000,141.000000,NaN,148.000000,141.000000,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,2.793415,2.468175,NaN,2.352941,2.260678,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,115.000000,127.000000,NaN,139.000000,138.000000,NaN,NaN,NaN


In [287]:
df = df[(df.Indicator == 'Availability of scientists and engineers')]
df = df[(df['Subindicator Type'] == '1-7 Best')]
df

,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,...,2008-2009,2009-2010,2010-2011,2011-2012,2012-2013,2013-2014,2014-2015,2015-2016,2016-2017,2017-2018
64,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,2.900664,2.377355,NaN,2.484848,2.487496,NaN,NaN,NaN
350,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.275245,3.229376,3.126520,3.165105,3.320792,3.510255,3.386487,3.238878,3.103442,3.280881
634,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,4.053544,4.743772,5.023698,4.947964,5.047487,5.099600,5.248989,5.248989,5.412238,5.443920
918,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.951912,3.885612,3.971024,3.976487,3.851021,3.885266,3.813951,3.495814,3.680224,3.942811
1204,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.985805,3.716545,3.659950,3.794084,4.002652,4.037141,3.970041,4.112069,4.244495,4.299045
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41800,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,4.489062,4.205438,4.099582,4.091871,4.015520,3.827813,3.810445,3.919625,3.770252,3.799701
42084,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,2.480131,2.691480,3.110059,3.138273,NaN,2.849202,3.155248
42368,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.365538,3.089414,3.272196,3.403178,3.363060,3.480697,3.540225,3.399498,3.396852,3.537765
42654,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,4.224592,3.982102,3.788823,3.912810,4.130981,4.181467,4.301357,4.419541,4.093391,3.803605


In [288]:
subpillars[1]
subpillar = subpillars[1]
print(subpillar)

Innovation Ecosystem


In [289]:
# create standard columns
# df.rename(columns={'COUNTRY/ECONOMY':'Country Name'}, inplace=True)

df['higher_is_better'] = True
df['Year'] = 2018
df['Indicator'] = indicator
df['data_col'] = df['2017-2018']
df['Sub-Pillar'] = subpillar

min_rank = df['data_col'].min()
max_rank = df['data_col'].max()

# transform 0-1 rank into 1-6
df['new_rank_score'] = df['data_col'].apply(lambda row: convert_rank_d(row,old_min=1,old_max=7))

In [290]:
df = df[['Country Name', 'Year','Indicator','data_col','new_rank_score','higher_is_better','Sub-Pillar']]
df

,Country Name,Year,Indicator,data_col,new_rank_score,higher_is_better,Sub-Pillar
64,Angola,2018,Availability of scientists and engineers,NaN,NaN,True,Innovation Ecosystem
350,Albania,2018,Availability of scientists and engineers,3.280881,2.900734,True,Innovation Ecosystem
634,United Arab Emirates,2018,Availability of scientists and engineers,5.443920,4.703267,True,Innovation Ecosystem
918,Argentina,2018,Availability of scientists and engineers,3.942811,3.452343,True,Innovation Ecosystem
1204,Armenia,2018,Availability of scientists and engineers,4.299045,3.749204,True,Innovation Ecosystem
...,...,...,...,...,...,...,...
41800,Vietnam,2018,Availability of scientists and engineers,3.799701,3.333084,True,Innovation Ecosystem
42084,"Yemen, Rep.",2018,Availability of scientists and engineers,3.155248,2.796040,True,Innovation Ecosystem
42368,South Africa,2018,Availability of scientists and engineers,3.537765,3.114804,True,Innovation Ecosystem
42654,Zambia,2018,Availability of scientists and engineers,3.803605,3.336338,True,Innovation Ecosystem


In [291]:
df.to_csv('../indicator_scores/infrastructure_{}_scores.csv'.format(indicator))

In [292]:
### 42. Scientific and technical articles/bn PPP$ GDP

In [293]:
indicators[41]

# load data
indicator = indicators[41]
print(indicator)
bf = bnames[bnames['Indicator']==indicator]['Filename'].values[0]
print(bf)

df = pd.read_csv('../../processed/{}.csv'.format(bf))

Scientific and technical articles/bn PPP$ GDP
global_innovation_dataset


In [294]:
df.head(15)

,ISO2,Economy,IndCode,IndNum,IndName,Rank,Score,Value,MinDatCov,Outdated,OverallStreWeak,IncomeStreWeak,DataYear
0,ALB,Albania,Inputs,.1,Innovation Input Sub-index,71.0,39.940929,NaN,NaN,NaN,NaN,NaN,NaN
1,ALB,Albania,Outputs,.2,Innovation Output Sub-index,92.0,16.109792,NaN,NaN,NaN,NaN,NaN,NaN
2,ALB,Albania,Index,0,Global Innovation Index,84.0,28.025361,NaN,NaN,NaN,NaN,NaN,NaN
3,ALB,Albania,P1,1,Institutions,60.0,64.919240,NaN,0.0,NaN,NaN,NaN,NaN
4,ALB,Albania,SP11,1.1.,Political environment,71.0,56.071183,NaN,0.0,NaN,NaN,NaN,NaN
5,ALB,Albania,PolStab,1.1.1,Political and operational stability*,60.0,69.642859,2.100000,NaN,0.0,NaN,NaN,2020.0
6,ALB,Albania,GovEff,1.1.2,Government effectiveness*,76.0,49.285344,-0.061331,NaN,0.0,NaN,NaN,2019.0
7,ALB,Albania,SP12,1.2.,Regulatory environment,82.0,58.941532,NaN,0.0,NaN,NaN,NaN,NaN
8,ALB,Albania,RegQua,1.2.1,Regulatory quality*,58.0,50.670701,0.274380,NaN,0.0,NaN,NaN,2019.0
9,ALB,Albania,RuleOL,1.2.2,Rule of law*,85.0,35.889077,-0.411179,NaN,0.0,NaN,NaN,2019.0


In [295]:
subpillars[1]
subpillar = subpillars[1]
print(subpillar)

Innovation Ecosystem


In [296]:
df = df[(df.IndName == 'Scientific and technical articles/bn PPP$ GDP')]
df

,ISO2,Economy,IndCode,IndNum,IndName,Rank,Score,Value,MinDatCov,Outdated,OverallStreWeak,IncomeStreWeak,DataYear
82,ALB,Albania,c_SciTechArt,6.1.4,Scientific and technical articles/bn PPP$ GDP,100.0,10.489509,7.210432,NaN,0.0,NaN,NaN,2020.0
194,DZA,Algeria,c_SciTechArt,6.1.4,Scientific and technical articles/bn PPP$ GDP,89.0,13.660362,9.258843,NaN,0.0,NaN,NaN,2020.0
306,AGO,Angola,c_SciTechArt,6.1.4,Scientific and technical articles/bn PPP$ GDP,131.0,0.000000,0.434080,NaN,0.0,W,W,2020.0
418,ARG,Argentina,c_SciTechArt,6.1.4,Scientific and technical articles/bn PPP$ GDP,76.0,16.693910,11.218552,NaN,0.0,NaN,NaN,2020.0
530,ARM,Armenia,c_SciTechArt,6.1.4,Scientific and technical articles/bn PPP$ GDP,43.0,32.230233,21.255209,NaN,0.0,NaN,NaN,2020.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
14306,UZB,Uzbekistan,c_SciTechArt,6.1.4,Scientific and technical articles/bn PPP$ GDP,125.0,2.570507,2.094659,NaN,0.0,W,NaN,2020.0
14418,VNM,Viet Nam,c_SciTechArt,6.1.4,Scientific and technical articles/bn PPP$ GDP,83.0,15.444373,10.411336,NaN,0.0,NaN,NaN,2020.0
14530,YEM,Yemen,c_SciTechArt,6.1.4,Scientific and technical articles/bn PPP$ GDP,81.0,15.693811,10.572476,NaN,0.0,S,NaN,2020.0
14642,ZMB,Zambia,c_SciTechArt,6.1.4,Scientific and technical articles/bn PPP$ GDP,95.0,13.007692,8.837210,NaN,0.0,NaN,NaN,2020.0


In [297]:
df['higher_is_better'] = True
df['Year'] = df['DataYear']
df['Indicator'] = indicator
df['data_col'] = df['Score']
df['Sub-Pillar'] = subpillar
df['Country Name'] = df['Economy']

min_rank = df['data_col'].min()
max_rank = df['data_col'].max()

# transform 0-1 rank into 1-6
df['new_rank_score'] = df['data_col'].apply(lambda row: convert_rank(row,old_min=min_rank,old_max=max_rank))

In [298]:
df = df[['Country Name', 'Year','Indicator','data_col','new_rank_score','higher_is_better','Sub-Pillar']]
df

,Country Name,Year,Indicator,data_col,new_rank_score,higher_is_better,Sub-Pillar
82,Albania,2020.0,Scientific and technical articles/bn PPP$ GDP,10.489509,1.524475,True,Innovation Ecosystem
194,Algeria,2020.0,Scientific and technical articles/bn PPP$ GDP,13.660362,1.683018,True,Innovation Ecosystem
306,Angola,2020.0,Scientific and technical articles/bn PPP$ GDP,0.000000,1.000000,True,Innovation Ecosystem
418,Argentina,2020.0,Scientific and technical articles/bn PPP$ GDP,16.693910,1.834695,True,Innovation Ecosystem
530,Armenia,2020.0,Scientific and technical articles/bn PPP$ GDP,32.230233,2.611512,True,Innovation Ecosystem
...,...,...,...,...,...,...,...
14306,Uzbekistan,2020.0,Scientific and technical articles/bn PPP$ GDP,2.570507,1.128525,True,Innovation Ecosystem
14418,Viet Nam,2020.0,Scientific and technical articles/bn PPP$ GDP,15.444373,1.772219,True,Innovation Ecosystem
14530,Yemen,2020.0,Scientific and technical articles/bn PPP$ GDP,15.693811,1.784691,True,Innovation Ecosystem
14642,Zambia,2020.0,Scientific and technical articles/bn PPP$ GDP,13.007692,1.650385,True,Innovation Ecosystem


In [299]:
df.to_csv('../indicator_scores/infrastructure_Scientific and technical articles_scores.csv')

In [300]:
### 43. Mobile app creation/bn PPP$ GDP

In [301]:
indicators[42]

# load data
indicator = indicators[42]
print(indicator)
bf = bnames[bnames['Indicator']==indicator]['Filename'].values[0]
print(bf)

df = pd.read_csv('../../processed/{}.csv'.format(bf))

Mobile app creation/bn PPP$ GDP
global_innovation_dataset


In [302]:
subpillars[1]
subpillar = subpillars[1]
print(subpillar)

Innovation Ecosystem


In [303]:
df = df[(df.IndName == 'Mobile app creation/bn PPP$ GDP')]
df

,ISO2,Economy,IndCode,IndNum,IndName,Rank,Score,Value,MinDatCov,Outdated,OverallStreWeak,IncomeStreWeak,DataYear
111,ALB,Albania,AppCrea,7.3.4,Mobile app creation/bn PPP$ GDP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
223,DZA,Algeria,AppCrea,7.3.4,Mobile app creation/bn PPP$ GDP,100.0,0.001067,0.000038,NaN,1.0,NaN,NaN,2019.0
335,AGO,Angola,AppCrea,7.3.4,Mobile app creation/bn PPP$ GDP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
447,ARG,Argentina,AppCrea,7.3.4,Mobile app creation/bn PPP$ GDP,52.0,8.357263,0.298769,NaN,0.0,NaN,NaN,2020.0
559,ARM,Armenia,AppCrea,7.3.4,Mobile app creation/bn PPP$ GDP,58.0,4.441897,0.158796,NaN,0.0,NaN,NaN,2020.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
14335,UZB,Uzbekistan,AppCrea,7.3.4,Mobile app creation/bn PPP$ GDP,99.0,0.003258,0.000117,NaN,0.0,W,NaN,2020.0
14447,VNM,Viet Nam,AppCrea,7.3.4,Mobile app creation/bn PPP$ GDP,10.0,47.909940,1.712761,NaN,0.0,S,S,2020.0
14559,YEM,Yemen,AppCrea,7.3.4,Mobile app creation/bn PPP$ GDP,84.0,0.185277,0.006624,NaN,0.0,NaN,S,2020.0
14671,ZMB,Zambia,AppCrea,7.3.4,Mobile app creation/bn PPP$ GDP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [304]:
df['higher_is_better'] = True
df['Year'] = df['DataYear']
df['Indicator'] = indicator
df['data_col'] = df['Score']
df['Sub-Pillar'] = subpillar
df['Country Name'] = df['Economy']

min_rank = df['data_col'].min()
max_rank = df['data_col'].max()

# transform 0-1 rank into 1-6
df['new_rank_score'] = df['data_col'].apply(lambda row: convert_rank(row,old_min=min_rank,old_max=max_rank))

In [305]:
df = df[['Country Name', 'Year','Indicator','data_col','new_rank_score','higher_is_better','Sub-Pillar']]
df

,Country Name,Year,Indicator,data_col,new_rank_score,higher_is_better,Sub-Pillar
111,Albania,NaN,Mobile app creation/bn PPP$ GDP,NaN,NaN,True,Innovation Ecosystem
223,Algeria,2019.0,Mobile app creation/bn PPP$ GDP,0.001067,1.000053,True,Innovation Ecosystem
335,Angola,NaN,Mobile app creation/bn PPP$ GDP,NaN,NaN,True,Innovation Ecosystem
447,Argentina,2020.0,Mobile app creation/bn PPP$ GDP,8.357263,1.417863,True,Innovation Ecosystem
559,Armenia,2020.0,Mobile app creation/bn PPP$ GDP,4.441897,1.222095,True,Innovation Ecosystem
...,...,...,...,...,...,...,...
14335,Uzbekistan,2020.0,Mobile app creation/bn PPP$ GDP,0.003258,1.000163,True,Innovation Ecosystem
14447,Viet Nam,2020.0,Mobile app creation/bn PPP$ GDP,47.909940,3.395497,True,Innovation Ecosystem
14559,Yemen,2020.0,Mobile app creation/bn PPP$ GDP,0.185277,1.009264,True,Innovation Ecosystem
14671,Zambia,NaN,Mobile app creation/bn PPP$ GDP,NaN,NaN,True,Innovation Ecosystem


In [306]:
df.to_csv('../indicator_scores/infrastructure_Mobile app creation_scores.csv')

In [307]:
### 44. Patents by origin/bn PPP$ GDP

In [308]:
indicators[43]

# load data
indicator = indicators[43]
print(indicator)
bf = bnames[bnames['Indicator']==indicator]['Filename'].values[0]
print(bf)

df = pd.read_csv('../../processed/{}.csv'.format(bf))

Patents by origin/bn PPP$ GDP
global_innovation_dataset


In [309]:
subpillars[1]
subpillar = subpillars[1]
print(subpillar)

Innovation Ecosystem


In [310]:
df = df[(df.IndName == 'Patents by origin/bn PPP$ GDP')]
df

,ISO2,Economy,IndCode,IndNum,IndName,Rank,Score,Value,MinDatCov,Outdated,OverallStreWeak,IncomeStreWeak,DataYear
79,ALB,Albania,PatOrig,6.1.1,Patents by origin/bn PPP$ GDP,106.0,1.081388,0.143850,NaN,0.0,NaN,NaN,2019.0
191,DZA,Algeria,PatOrig,6.1.1,Patents by origin/bn PPP$ GDP,96.0,1.667888,0.221869,NaN,0.0,NaN,NaN,2019.0
303,AGO,Angola,PatOrig,6.1.1,Patents by origin/bn PPP$ GDP,127.0,0.067585,0.008990,NaN,0.0,NaN,NaN,2019.0
415,ARG,Argentina,PatOrig,6.1.1,Patents by origin/bn PPP$ GDP,82.0,3.215135,0.427690,NaN,0.0,NaN,NaN,2019.0
527,ARM,Armenia,PatOrig,6.1.1,Patents by origin/bn PPP$ GDP,28.0,20.891727,2.779097,NaN,0.0,S,NaN,2019.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
14303,UZB,Uzbekistan,PatOrig,6.1.1,Patents by origin/bn PPP$ GDP,47.0,11.477493,1.526780,NaN,0.0,NaN,NaN,2019.0
14415,VNM,Viet Nam,PatOrig,6.1.1,Patents by origin/bn PPP$ GDP,73.0,5.324811,0.708327,NaN,0.0,NaN,NaN,2019.0
14527,YEM,Yemen,PatOrig,6.1.1,Patents by origin/bn PPP$ GDP,66.0,6.929591,0.921801,NaN,0.0,S,S,2019.0
14639,ZMB,Zambia,PatOrig,6.1.1,Patents by origin/bn PPP$ GDP,123.0,0.232702,0.030955,NaN,0.0,W,NaN,2019.0


In [311]:
df['higher_is_better'] = True
df['Year'] = df['DataYear']
df['Indicator'] = indicator
df['data_col'] = df['Score']
df['Sub-Pillar'] = subpillar
df['Country Name'] = df['Economy']

min_rank = df['data_col'].min()
max_rank = df['data_col'].max()

# transform 0-1 rank into 1-6
df['new_rank_score'] = df['data_col'].apply(lambda row: convert_rank(row,old_min=min_rank,old_max=max_rank))

In [312]:
df = df[['Country Name', 'Year','Indicator','data_col','new_rank_score','higher_is_better','Sub-Pillar']]
df

,Country Name,Year,Indicator,data_col,new_rank_score,higher_is_better,Sub-Pillar
79,Albania,2019.0,Patents by origin/bn PPP$ GDP,1.081388,1.054069,True,Innovation Ecosystem
191,Algeria,2019.0,Patents by origin/bn PPP$ GDP,1.667888,1.083394,True,Innovation Ecosystem
303,Angola,2019.0,Patents by origin/bn PPP$ GDP,0.067585,1.003379,True,Innovation Ecosystem
415,Argentina,2019.0,Patents by origin/bn PPP$ GDP,3.215135,1.160757,True,Innovation Ecosystem
527,Armenia,2019.0,Patents by origin/bn PPP$ GDP,20.891727,2.044586,True,Innovation Ecosystem
...,...,...,...,...,...,...,...
14303,Uzbekistan,2019.0,Patents by origin/bn PPP$ GDP,11.477493,1.573875,True,Innovation Ecosystem
14415,Viet Nam,2019.0,Patents by origin/bn PPP$ GDP,5.324811,1.266241,True,Innovation Ecosystem
14527,Yemen,2019.0,Patents by origin/bn PPP$ GDP,6.929591,1.346480,True,Innovation Ecosystem
14639,Zambia,2019.0,Patents by origin/bn PPP$ GDP,0.232702,1.011635,True,Innovation Ecosystem


In [313]:
df.to_csv('../indicator_scores/infrastructure_Patents by origin_scores.csv')

In [314]:
### 45. International patent applications by origin (PCT System)

In [315]:
indicators[44]

# load data
indicator = indicators[44]
print(indicator)
bf = bnames[bnames['Indicator']==indicator]['Filename'].values[0]
print(bf)

df = pd.read_csv('../../processed/{}.csv'.format(bf))

International patent applications by origin (PCT System)
patent_application_PCT


In [316]:
df.head(15)

,2018,Origin,2019 Estimate
0,53349,China,58990
1,56252,United States of America,57840
2,49706,Japan,52660
3,19742,Germany,19353
4,16917,Republic of Korea,19085
5,7918,France,7934
6,5634,United Kingdom,5786
7,4576,Switzerland,4610
8,4168,Sweden,4185
9,4134,Netherlands,4011


In [317]:
subpillars[1]
subpillar = subpillars[1]
print(subpillar)

Innovation Ecosystem


In [318]:
df['higher_is_better'] = True
df['Year'] = 2019
df['Indicator'] = indicator
df['data_col'] = df['2019 Estimate']
df['Sub-Pillar'] = subpillar
df['Country Name'] = df['Origin']

min_rank = df['data_col'].min()
max_rank = df['data_col'].max()

# transform 0-1 rank into 1-6
df['new_rank_score'] = df['data_col'].apply(lambda row: convert_rank(row,old_min=min_rank,old_max=max_rank))

In [319]:
df = df[['Country Name', 'Year','Indicator','data_col','new_rank_score','higher_is_better','Sub-Pillar']]
df

,Country Name,Year,Indicator,data_col,new_rank_score,higher_is_better,Sub-Pillar
0,China,2019,International patent applications by origin (P...,58990,6.000000,True,Innovation Ecosystem
1,United States of America,2019,International patent applications by origin (P...,57840,5.902526,True,Innovation Ecosystem
2,Japan,2019,International patent applications by origin (P...,52660,5.463468,True,Innovation Ecosystem
3,Germany,2019,International patent applications by origin (P...,19353,2.640363,True,Innovation Ecosystem
4,Republic of Korea,2019,International patent applications by origin (P...,19085,2.617647,True,Innovation Ecosystem
...,...,...,...,...,...,...,...
139,Tajikistan,2019,International patent applications by origin (P...,0,1.000000,True,Innovation Ecosystem
140,United Republic of Tanzania,2019,International patent applications by origin (P...,0,1.000000,True,Innovation Ecosystem
141,Yemen,2019,International patent applications by origin (P...,0,1.000000,True,Innovation Ecosystem
142,Zambia,2019,International patent applications by origin (P...,0,1.000000,True,Innovation Ecosystem


In [320]:
### 46. International industrial applications by origin (Hague System)

In [321]:
indicators[45]

# load data
indicator = indicators[45]
print(indicator)
bf = bnames[bnames['Indicator']==indicator]['Filename'].values[0]
print(bf)

df = pd.read_csv('../../processed/{}.csv'.format(bf))
df.head(15)

International industrial applications by origin (Hague System)
Hague_system


,Origin,Applications - 2018,Applications - 2019,Designs - 2018,Designs - 2019
0,Germany,713,772,3964,4487
1,Republic of Korea,922,1176,1545,2736
2,Switzerland,648,601,2453,2178
3,Italy,324,412,1261,1994
4,Netherlands,179,181,1352,1376
5,United States of America,369,410,1665,1351
6,France,466,467,1437,1298
7,Japan,401,397,1259,1152
8,China,139,239,315,663
9,United Kingdom,136,182,372,548


In [322]:
subpillars[1]
subpillar = subpillars[1]
print(subpillar)

Innovation Ecosystem


In [323]:
df['higher_is_better'] = True
df['Year'] = 2019
df['Indicator'] = indicator
df['data_col'] = df.iloc[:,2]
df['Sub-Pillar'] = subpillar
df['Country Name'] = df['Origin']

min_rank = df['data_col'].min()
max_rank = df['data_col'].max()

# transform 0-1 rank into 1-6
df['new_rank_score'] = df['data_col'].apply(lambda row: convert_rank(row,old_min=min_rank,old_max=max_rank))

In [324]:
df = df[['Country Name', 'Year','Indicator','data_col','new_rank_score','higher_is_better','Sub-Pillar']]
df

,Country Name,Year,Indicator,data_col,new_rank_score,higher_is_better,Sub-Pillar
0,Germany,2019,International industrial applications by origi...,772,4.282313,True,Innovation Ecosystem
1,Republic of Korea,2019,International industrial applications by origi...,1176,6.000000,True,Innovation Ecosystem
2,Switzerland,2019,International industrial applications by origi...,601,3.555272,True,Innovation Ecosystem
3,Italy,2019,International industrial applications by origi...,412,2.751701,True,Innovation Ecosystem
4,Netherlands,2019,International industrial applications by origi...,181,1.769558,True,Innovation Ecosystem
...,...,...,...,...,...,...,...
69,Malaysia,2019,International industrial applications by origi...,0,1.000000,True,Innovation Ecosystem
70,Malta,2019,International industrial applications by origi...,0,1.000000,True,Innovation Ecosystem
71,Seychelles,2019,International industrial applications by origi...,0,1.000000,True,Innovation Ecosystem
72,Turkmenistan,2019,International industrial applications by origi...,0,1.000000,True,Innovation Ecosystem


In [325]:
### 47. International industrial designs by origin (Hague System)

In [326]:
indicators[46]

# load data
indicator = indicators[46]
print(indicator)
bf = bnames[bnames['Indicator']==indicator]['Filename'].values[0]
print(bf)

df = pd.read_csv('../../processed/{}.csv'.format(bf))
df.head(15)

International industrial designs by origin (Hague System)
Hague_system


,Origin,Applications - 2018,Applications - 2019,Designs - 2018,Designs - 2019
0,Germany,713,772,3964,4487
1,Republic of Korea,922,1176,1545,2736
2,Switzerland,648,601,2453,2178
3,Italy,324,412,1261,1994
4,Netherlands,179,181,1352,1376
5,United States of America,369,410,1665,1351
6,France,466,467,1437,1298
7,Japan,401,397,1259,1152
8,China,139,239,315,663
9,United Kingdom,136,182,372,548


In [327]:
subpillars[1]
subpillar = subpillars[1]
print(subpillar)

Innovation Ecosystem


In [328]:
df['higher_is_better'] = True
df['Year'] = 2019
df['Indicator'] = indicator
df['data_col'] = df.iloc[:,4]
df['Sub-Pillar'] = subpillar
df['Country Name'] = df['Origin']

min_rank = df['data_col'].min()
max_rank = df['data_col'].max()

# transform 0-1 rank into 1-6
df['new_rank_score'] = df['data_col'].apply(lambda row: convert_rank(row,old_min=min_rank,old_max=max_rank))

In [329]:
df = df[['Country Name', 'Year','Indicator','data_col','new_rank_score','higher_is_better','Sub-Pillar']]
df

,Country Name,Year,Indicator,data_col,new_rank_score,higher_is_better,Sub-Pillar
0,Germany,2019,International industrial designs by origin (Ha...,4487,6.000000,True,Innovation Ecosystem
1,Republic of Korea,2019,International industrial designs by origin (Ha...,2736,4.048808,True,Innovation Ecosystem
2,Switzerland,2019,International industrial designs by origin (Ha...,2178,3.427011,True,Innovation Ecosystem
3,Italy,2019,International industrial designs by origin (Ha...,1994,3.221975,True,Innovation Ecosystem
4,Netherlands,2019,International industrial designs by origin (Ha...,1376,2.533318,True,Innovation Ecosystem
...,...,...,...,...,...,...,...
69,Malaysia,2019,International industrial designs by origin (Ha...,0,1.000000,True,Innovation Ecosystem
70,Malta,2019,International industrial designs by origin (Ha...,0,1.000000,True,Innovation Ecosystem
71,Seychelles,2019,International industrial designs by origin (Ha...,0,1.000000,True,Innovation Ecosystem
72,Turkmenistan,2019,International industrial designs by origin (Ha...,0,1.000000,True,Innovation Ecosystem


In [330]:
### Score Aggregating

In [331]:
import os

In [332]:
# get list of files in scores folder
scores = os.listdir('../indicator_scores/')
scores = [s for s in scores if s.startswith('infrastructure')]

In [333]:
scores

['infrastructure_% of population covered by electricity_scores.csv',
 'infrastructure_% of population covered by internet connectivity_scores.csv',
 'infrastructure_% of population covered by mobile 2G+ data connectivity_scores.csv',
 'infrastructure_% of population covered by mobile 3G+ data connectivity_scores.csv',
 'infrastructure_% of population covered by mobile 4G+ data connectivity_scores.csv',
 'infrastructure_% of population covered by mobile 5G+ data connectivity_scores.csv',
 'infrastructure_Availability of scientists and engineers_scores.csv',
 'infrastructure_Broadband Density_scores.csv',
 'infrastructure_Cellphone Signal Density_scores.csv',
 'infrastructure_Digital Finance Ecosystem size_scores.csv',
 'infrastructure_Digital Work Ecosystem size_scores.csv',
 'infrastructure_Electricity Density_scores.csv',
 'infrastructure_Electricity Environmental Sustainability_scores.csv',
 'infrastructure_Electricity supply quality_scores.csv',
 'infrastructure_Global Cybersecurity

In [334]:
# create a dataframe that concatenates all these file into one table
df = pd.concat([pd.read_csv('../indicator_scores/{}'.format(s)) for s in scores])    

In [335]:
df

,Country Name,Year,Indicator,data_col,new_rank_score,higher_is_better,Sub-Pillar,Unnamed: 0
0,Aruba,2019.0,% of population covered by electricity,100.000000,6.000000,True,Connectivity Technology,NaN
1,Africa Eastern and Southern,2019.0,% of population covered by electricity,43.640661,3.182033,True,Connectivity Technology,NaN
2,Afghanistan,2019.0,% of population covered by electricity,97.700000,5.885000,True,Connectivity Technology,NaN
3,Africa Western and Central,2019.0,% of population covered by electricity,51.341421,3.567071,True,Connectivity Technology,NaN
4,Angola,2019.0,% of population covered by electricity,45.670315,3.283516,True,Connectivity Technology,NaN
...,...,...,...,...,...,...,...,...
188,Senegal,2020.0,Telecommunication Infrastructure Index (TII),0.435800,3.179000,True,Connectivity Technology,NaN
189,Serbia,2020.0,Telecommunication Infrastructure Index (TII),0.620000,4.100000,True,Connectivity Technology,NaN
190,Seychelles,2020.0,Telecommunication Infrastructure Index (TII),0.692500,4.462500,True,Connectivity Technology,NaN
191,Singapore,2020.0,Telecommunication Infrastructure Index (TII),0.889900,5.449500,True,Connectivity Technology,NaN


In [336]:
# Data cleaning
df['new_rank_score'] = df['new_rank_score'].fillna(0)
df.sort_values(by=['Country Name'], ascending=True, inplace=True)
df.reset_index(drop=True, inplace=True)

# remove trailing whitespaces from country name
df['Country Name'] = df['Country Name'].str.strip()
df['Country Name'] = df['Country Name'].str.strip('**')
df['Country Name'] = df['Country Name'].str.strip('*')


# Replace values that are not truly country names with nan
df['Country Name'] = df['Country Name'].replace('n.a. : non avalaible',np.nan)
df['Country Name'] = df['Country Name'].replace('nan',np.nan)
df['Country Name'] = df['Country Name'].replace('Not classified',np.nan)
df['Country Name'] = df['Country Name'].replace('Source :',np.nan)
df['Country Name'] = df['Country Name'].replace('© Copyright Enerdata. Reproduction and diffusion prohibited (web, photocopy, intranet...) without written permission.',np.nan)

# Dropping the columns having NaN/NaT values
df = df[df['Country Name'].notna()]


df['Country Name'] = df['Country Name'].astype(str, errors = 'ignore')



C:\Users\OKRSTU~1\AppData\Local\Temp/ipykernel_6400/1251001374.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Country Name'] = df['Country Name'].astype(str, errors = 'ignore')


In [337]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4899 entries, 0 to 4899
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Country Name      4899 non-null   object 
 1   Year              4869 non-null   float64
 2   Indicator         4899 non-null   object 
 3   data_col          4487 non-null   float64
 4   new_rank_score    4899 non-null   float64
 5   higher_is_better  4899 non-null   bool   
 6   Sub-Pillar        4899 non-null   object 
 7   Unnamed: 0        808 non-null    float64
dtypes: bool(1), float64(4), object(3)
memory usage: 311.0+ KB


In [338]:
df.head(15)

,Country Name,Year,Indicator,data_col,new_rank_score,higher_is_better,Sub-Pillar,Unnamed: 0
0,Afghanistan,2019.0,Postal Coverage,50.000000,3.500000,True,Connectivity Technology,NaN
1,Afghanistan,2019.0,Electricity Density,27.128000,1.002387,True,Connectivity Technology,NaN
2,Afghanistan,2019.0,GSMA Mobile Connectivity Index,28.940000,2.447000,True,Connectivity Technology,NaN
3,Afghanistan,2019.0,% of population covered by mobile 5G+ data con...,0.000000,1.000000,True,Connectivity Technology,NaN
4,Afghanistan,2020.0,Broadband Density,0.068254,1.003413,True,Connectivity Technology,36853.0
5,Afghanistan,2020.0,Global Cybersecurity Index (GCI),5.200000,1.260000,True,Connectivity Technology,NaN
6,Afghanistan,2020.0,Telecommunication Infrastructure Index (TII),0.176200,1.881000,True,Connectivity Technology,NaN
7,Afghanistan,2019.0,% of population covered by mobile 3G+ data con...,58.700000,3.935000,True,Connectivity Technology,NaN
8,Afghanistan,2021.0,National cyber security index,11.690000,1.584500,True,Connectivity Technology,NaN
9,Afghanistan,2019.0,% of population covered by mobile 2G+ data con...,90.000000,5.500000,True,Connectivity Technology,NaN


In [339]:
sorted(df['Country Name'].unique().tolist())

['Afghanistan',
 'Africa',
 'Africa Eastern and Southern',
 'Africa Western and Central',
 'Albania',
 'Algeria',
 'American Samoa',
 'Andorra',
 'Angola',
 'Antigua and Barbuda',
 'Arab World',
 'Argentina',
 'Armenia',
 'Aruba',
 'Asia Pacific',
 'Australia',
 'Austria',
 'Azerbaijan',
 'Bahamas',
 'Bahamas, The',
 'Bahrain',
 'Bahrain (Kingdom of)',
 'Bangladesh',
 'Barbados',
 'Belarus',
 'Belgium',
 'Belgium Estonia',
 'Belize',
 'Benin',
 'Bermuda',
 'Bhutan',
 'Bolivia',
 'Bolivia (Plurinational State of)',
 'Bosnia and Herzegovina',
 'Botswana',
 'Brazil',
 'British Virgin Islands',
 'Brunei',
 'Brunei Darussalam',
 'Bulgaria',
 'Bulgaria (Rep.)',
 'Burkina Faso',
 'Burundi',
 'CIS',
 'Cabo Verde',
 'Cambodia',
 'Cameroon',
 'Canada',
 'Cape Verde',
 'Caribbean small states',
 'Cayman Islands',
 'Central African Rep.',
 'Central African Republic',
 'Central America',
 'Central Europe and the Baltics',
 'Chad',
 'Channel Islands',
 'Chile',
 'China',
 "China (People's Rep.)",
 '

In [340]:
df.head(15)

,Country Name,Year,Indicator,data_col,new_rank_score,higher_is_better,Sub-Pillar,Unnamed: 0
0,Afghanistan,2019.0,Postal Coverage,50.000000,3.500000,True,Connectivity Technology,NaN
1,Afghanistan,2019.0,Electricity Density,27.128000,1.002387,True,Connectivity Technology,NaN
2,Afghanistan,2019.0,GSMA Mobile Connectivity Index,28.940000,2.447000,True,Connectivity Technology,NaN
3,Afghanistan,2019.0,% of population covered by mobile 5G+ data con...,0.000000,1.000000,True,Connectivity Technology,NaN
4,Afghanistan,2020.0,Broadband Density,0.068254,1.003413,True,Connectivity Technology,36853.0
5,Afghanistan,2020.0,Global Cybersecurity Index (GCI),5.200000,1.260000,True,Connectivity Technology,NaN
6,Afghanistan,2020.0,Telecommunication Infrastructure Index (TII),0.176200,1.881000,True,Connectivity Technology,NaN
7,Afghanistan,2019.0,% of population covered by mobile 3G+ data con...,58.700000,3.935000,True,Connectivity Technology,NaN
8,Afghanistan,2021.0,National cyber security index,11.690000,1.584500,True,Connectivity Technology,NaN
9,Afghanistan,2019.0,% of population covered by mobile 2G+ data con...,90.000000,5.500000,True,Connectivity Technology,NaN


In [341]:
# average indicator scores per country
agg_df = df.groupby(['Country Name']).agg({'new_rank_score':'mean','data_col':'count'})

In [342]:
agg_df.columns = ['agg_score', 'count_source' ]

In [343]:
max_number_sources = agg_df.describe()['count_source']['max']

In [344]:
agg_df['agg_score_wt'] = agg_df['agg_score']*(agg_df['count_source']/max_number_sources)

In [345]:
agg_df.sort_values(by='agg_score', ascending=False, inplace=True)

In [346]:
agg_df.head(25)

,agg_score,count_source,agg_score_wt
Country Name,,,
St. Martin (French part),6.000000,1,0.200000
Sint Maarten (Dutch part),6.000000,1,0.200000
"Hongkong, China",6.000000,1,0.200000
Czech Rep.,6.000000,1,0.200000
"Macao SAR, China",6.000000,1,0.200000
Isle of Man,6.000000,1,0.200000
Curacao,6.000000,1,0.200000
Jersey,6.000000,1,0.200000
Korea (Rep.),6.000000,1,0.200000


In [347]:
agg_df.to_csv('../pillar_scores/infrastructure_scores_v0.csv')

In [348]:
### Score Aggregating by Subpillars

In [349]:
df.insert(0,'Pillar','Infrastructure')
df

# remove trailing whitespaces from country name
df['Country Name'] = df['Country Name'].str.strip()
df['Country Name'] = df['Country Name'].str.strip('**')
df['Country Name'] = df['Country Name'].str.strip('*')

C:\Users\OKRSTU~1\AppData\Local\Temp/ipykernel_6400/1902140528.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Country Name'] = df['Country Name'].str.strip()
C:\Users\OKRSTU~1\AppData\Local\Temp/ipykernel_6400/1902140528.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Country Name'] = df['Country Name'].str.strip('**')
C:\Users\OKRSTU~1\AppData\Local\Temp/ipykernel_6400/1902140528.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .lo

In [350]:
sub_df = df.groupby(['Pillar','Sub-Pillar','Country Name']).agg({'new_rank_score':'mean','data_col':'count'})

In [351]:
sub_df.columns = ['agg_score', 'count_source' ]

In [352]:
max_number_sources = sub_df.describe()['count_source']['max']

In [353]:
sub_df['agg_score_wt'] = sub_df['agg_score']*(sub_df['count_source']/max_number_sources)

In [354]:
sub_df.to_csv('../subpillar_score/infrastructure_scores_subpillar_v0.csv')

### Sources Generation

In [355]:
#Get all countries from Countries.xlsx
countries = pd.read_excel('../../data/Countries.xlsx')
col_names = ['Country or Area']
countries = countries[col_names]
countries.rename(columns = {'Country or Area': 'Country Name'}, inplace = True)

In [356]:
#Get all indicators from names dataframe retrieve at the begining of the script
bnames=bnames[['check','Sub-Pillar','Indicator','Data Source','Data Link']]
bnames.rename(columns = {'check': 'Pillar'}, inplace = True)
bnames = bnames.replace('\n','', regex=True)

In [357]:
#Do a nice cross join so that we have combination of all countries vs all indicators
sources = countries.merge(bnames, how='cross')
sources

,Country Name,Pillar,Sub-Pillar,Indicator,Data Source,Data Link
0,Algeria,Infrastructure,Connectivity Technology,Population Density,World Bank: World Development Indicators,https://datacatalog.worldbank.org
1,Algeria,Infrastructure,Connectivity Technology,Broadband Density,ITU: Digital Development Dashboard,https://www.itu.int/itu-d/sites/statistics
2,Algeria,Infrastructure,Connectivity Technology,% of population covered by internet connectivity,ITU: Digital Development Dashboard,https://www.itu.int/itu-d/sites/statistics
3,Algeria,Infrastructure,Connectivity Technology,% of population covered by mobile 2G+ data con...,GSMA: Mobile Connectivity Index,https://www.mobileconnectivityindex.com
4,Algeria,Infrastructure,Connectivity Technology,% of population covered by mobile 3G+ data con...,GSMA: Mobile Connectivity Index,https://www.mobileconnectivityindex.com
...,...,...,...,...,...,...
11698,Wallis and Futuna Islands,Infrastructure,Innovation Ecosystem,Mobile app creation/bn PPP$ GDP,Global Innovation Index/App Annie Intelligence...,https://www.globalinnovationindex.org/analysis...
11699,Wallis and Futuna Islands,Infrastructure,Innovation Ecosystem,Patents by origin/bn PPP$ GDP,Global Innovation Index/WIPO: Patents by origi...,https://www.globalinnovationindex.org/analysis...
11700,Wallis and Futuna Islands,Infrastructure,Innovation Ecosystem,International patent applications by origin (P...,WIPO: Intellectual Property Statistics,https://www.wipo.int/ipstats/en
11701,Wallis and Futuna Islands,Infrastructure,Innovation Ecosystem,International industrial applications by origi...,WIPO: Intellectual Property Statistics,https://www.wipo.int/ipstats/en


In [358]:
#Make copy of a scores dataframe and add the column available, with value of 1 (string)
#denoting all the country/indicator combinations that have value
dfsources = df[['Country Name','Pillar','Sub-Pillar','Indicator']].copy()
dfsources['Available'] = '1'

In [359]:
#Merge (left join) sources (all country indicator combinations)
#with those in copied scores dataframe
#resulting in a sources dataframe with values of 1 in Available column where there is a value for country/indicator
#and there is a NaN where there's not. 
#subsequently replace NaN with 0 (string)
sources = sources.merge(dfsources, how='left', on=['Country Name','Pillar','Sub-Pillar','Indicator'])
sources['Available'] = sources['Available'].fillna('0')
sources

,Country Name,Pillar,Sub-Pillar,Indicator,Data Source,Data Link,Available
0,Algeria,Infrastructure,Connectivity Technology,Population Density,World Bank: World Development Indicators,https://datacatalog.worldbank.org,0
1,Algeria,Infrastructure,Connectivity Technology,Broadband Density,ITU: Digital Development Dashboard,https://www.itu.int/itu-d/sites/statistics,1
2,Algeria,Infrastructure,Connectivity Technology,% of population covered by internet connectivity,ITU: Digital Development Dashboard,https://www.itu.int/itu-d/sites/statistics,0
3,Algeria,Infrastructure,Connectivity Technology,% of population covered by mobile 2G+ data con...,GSMA: Mobile Connectivity Index,https://www.mobileconnectivityindex.com,1
4,Algeria,Infrastructure,Connectivity Technology,% of population covered by mobile 3G+ data con...,GSMA: Mobile Connectivity Index,https://www.mobileconnectivityindex.com,1
...,...,...,...,...,...,...,...
11698,Wallis and Futuna Islands,Infrastructure,Innovation Ecosystem,Mobile app creation/bn PPP$ GDP,Global Innovation Index/App Annie Intelligence...,https://www.globalinnovationindex.org/analysis...,0
11699,Wallis and Futuna Islands,Infrastructure,Innovation Ecosystem,Patents by origin/bn PPP$ GDP,Global Innovation Index/WIPO: Patents by origi...,https://www.globalinnovationindex.org/analysis...,0
11700,Wallis and Futuna Islands,Infrastructure,Innovation Ecosystem,International patent applications by origin (P...,WIPO: Intellectual Property Statistics,https://www.wipo.int/ipstats/en,0
11701,Wallis and Futuna Islands,Infrastructure,Innovation Ecosystem,International industrial applications by origi...,WIPO: Intellectual Property Statistics,https://www.wipo.int/ipstats/en,0


In [360]:
#If sources.csv exists, get the contents, remove everyhing from this pillar, append prepared sources, save csv.
#if sources.csv does not exist, create new file from sources.
from os.path import exists

if exists('../../dashboard/Sources.csv') :
    CurrentSources = pd.read_csv('../../dashboard/Sources.csv', dtype=str)
    CurrentSources = CurrentSources[['Country Name','Pillar','Sub-Pillar','Indicator','Data Source','Data Link','Available']]
    CurrentSources = CurrentSources.loc[CurrentSources['Pillar'] != 'Infrastructure']
    CurrentSources = CurrentSources.append(sources)
else :
    CurrentSources = sources
CurrentSources

,Country Name,Pillar,Sub-Pillar,Indicator,Data Source,Data Link,Available
0,Algeria,Government,Digital Public Services,Online-Service-Index (OSI),UN: E-Government Survey,https://publicadministration.un.org/egovkb,NaN
1,Algeria,Government,Digital Public Services,E-Participation index,UN: E-Government Survey,https://publicadministration.un.org/egovkb,NaN
2,Algeria,Government,Digital Public Services,Use of public services online (% of services o...,Boston Consulting Group/Salesforce: The Global...,https://www.salesforce.com,NaN
3,Algeria,Government,Digital Public Services,Security incidents (# of relevant issues),SPECOPS,https://specopssoft.com,NaN
4,Algeria,Government,Digital Public Services,What is the % change of government digitizing ...,World Bank: GovTech Dataset,https://datacatalog.worldbank.org,NaN
...,...,...,...,...,...,...,...
11698,Wallis and Futuna Islands,Infrastructure,Innovation Ecosystem,Mobile app creation/bn PPP$ GDP,Global Innovation Index/App Annie Intelligence...,https://www.globalinnovationindex.org/analysis...,0
11699,Wallis and Futuna Islands,Infrastructure,Innovation Ecosystem,Patents by origin/bn PPP$ GDP,Global Innovation Index/WIPO: Patents by origi...,https://www.globalinnovationindex.org/analysis...,0
11700,Wallis and Futuna Islands,Infrastructure,Innovation Ecosystem,International patent applications by origin (P...,WIPO: Intellectual Property Statistics,https://www.wipo.int/ipstats/en,0
11701,Wallis and Futuna Islands,Infrastructure,Innovation Ecosystem,International industrial applications by origi...,WIPO: Intellectual Property Statistics,https://www.wipo.int/ipstats/en,0


In [361]:
CurrentSources.to_csv('../../dashboard/Sources.csv', index=False)